# PROCESO FINAL B

# Multiva

In [9]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime 

fecha=str(datetime.date.today())

fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\MULTIVA\\MULTIVAFijos.csv")
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\MULTIVA\\"+fecha+"\\DicisMULTIVA")
docs=os.listdir() 
error=[]

for p in docs:
    try:
        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\MULTIVA\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\MULTIVA\\"+fecha+"\\DicisMULTIVA\\"+p
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\MULTIVA\\Templates\MULTIVA_Template.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\MULTIVA\\Templates\MULTIVA_TemplateFull.json"

        pdf = PyPDF2.PdfFileReader(rutapdf)

                #replicamos nuestro template para todas las series
        full_temp = list()
        for n_pags in range((pdf.getNumPages())//2):
            with open(rutatemplate) as f:
                temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

                            #guardamos el nuevo template
        with open("MULTIVA_TemplateFull.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutafull)

        razon=" ".join(data[1].T.reset_index().T.dropna().iloc[:,0])
        indexa=" ".join(data[0].T.reset_index().iloc[1:,2].dropna())
        fechaux=" ".join(data[0].T.reset_index().iloc[1:,4].dropna())

        if("con oficio" in fechaux.lower()):
            fechaux=fechaux[:fechaux.lower().find("con o")].strip()

        try:
            dia=fechaux[:fechaux.find("/")]
            mes=fechaux[fechaux.find("/")+1:][:fechaux[fechaux.find("/")+1:].find("/")]
            año=fechaux[fechaux.find("/")+1:][fechaux[fechaux.find("/")+1:].find("/")+1:]

            if ( len(dia)==1 ):
                dia="0"+dia
            if ( len(mes)==1 ):
                mes="0"+mes
            fechaux="20"+año+mes+dia

        except:
            fechaux=fechaux

        auxcal=" ".join(data[0].T.reset_index().iloc[1:,3].dropna())
        try:
            if ("homogénea" in auxcal.lower()):
                auxcal=auxcal[auxcal.find("énea")+4:auxcal.find("/")+2].strip()
            elif("fitch" in auxcal.lower()):
                auxcal=auxcal[auxcal.lower().find("fitch")+5:].strip().replace("f","").replace("F","").replace("s","").replace("S","")
            else:
                auxcal=auxcal
        except:
            auxcal=auxcal

        auxhora=' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
        try:
            auxhora=auxhora[auxhora.find("a")+1:].strip()
        except:
            auxhora=auxhora
            
        if "variable" in " ".join(data[0].T.reset_index().iloc[1:,0].dropna()).lower() :
            tipoaux=101
        elif "deuda" in " ".join(data[0].T.reset_index().iloc[1:,0].dropna()).lower() :
            tipoaux=100 
        
        auxperm=" ".join(data[6].T.reset_index().T.iloc[:,1].dropna()).replace("Unnamed: 0","").strip()
        #if "depende" in auxperm.lower():
         #   auxperm="No existe"
        RES={
                                  "fondo": {
                                        "nombre":" ".join(data[0].T.reset_index().iloc[1:,2].dropna()),
                                        "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                                        "razon_social":{
                                            "valor":razon[razon.lower().find("social")+7:razon.lower().find("adminis")].strip()
                                        },
                                        "fecha_autorizacion": {
                                            "valor":fechaux
                                        },
                                        "tipo": {
                                            "valor":" ".join(data[0].T.reset_index().iloc[1:,0].dropna()),
                                            "id": tipoaux
                                        },
                                        "administradora": {
                                            "nombre":razon[razon.lower().find(" por ")+4:].strip(),
                                            "valor": 9
                                        },
                                        "horizonte_inversion": {
                                            "valor": fijos.iloc[1,list(fijos.columns).index(indexa)]
                                        },
                                        "liquidez": {
                                            "valor": ' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,0].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                        },
                                        "permanencia": {
                                            "valor":auxperm
                                        },
                                        "calificacion_mercado": {
                                            "valor": auxcal
                                        },
                                        "director": {
                                            "valor": fijos.iloc[2,list(fijos.columns).index(indexa)]
                                        },
                                        "vigente": 1,
                                        "categoria": {
                                            "valor": " ".join(data[0].T.reset_index().iloc[1:,1].dropna()).replace("\r"," ")
                                        },
                                        "horario_cierre": {
                                            "valor":auxhora,
                                        },
                                        "campos_faltantes": [],
                                        "serie":[
                                        ]
                                    }
                                }
        aux=[]

        for i in range((pdf.getNumPages())//2):
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[3:,0].dropna())
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[3:,1].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[3:,2].dropna())
            comicliente=data[(7* i)+3].T.reset_index().iloc[:,0][1]
            comifondo=data[(7* i)+4].T.reset_index().iloc[:,0][1]

            aux.append({"nombre": serie.strip().replace(".",""),
                                                 "monto_minimo": {
                                                     "valor": limites
                                                },
                                                 "monto_maximo": {
                                                     "valor": None
                                                },
                                                 "comision_cliente": {
                                                     "valor": comicliente
                                                },
                                                  "comision_fondo": {
                                                     "valor": comifondo
                                                },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui.strip().replace(".","")
                                                },
                                                 "vigente":1
                                                ,
                                                 "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                "campos_faltantes": []
                                                })
        RES["fondo"]["serie"]=aux  


        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\MULTIVA\Jsons\\"+fecha)
        with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"success {p}")
    except Exception as e:
        print(f"error en {p}")
        print(e)
        error.append()

success MULTIAR.pdf
success MULTIED.pdf
success MULTIEQ.pdf
success MULTIFA.pdf
success MULTINS.pdf
success MULTIPC.pdf
success MULTIRE.pdf
success MULTISI.pdf
success MULTIUS.pdf
success MVJER.pdf


## BNP

In [35]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime 

fecha=str(datetime.date.today())
os.chdir("C:\\Users\\ortca\\Analysic_Nabla\\Proyectos\\INFOSEL\\ProcesoB\Fondos_root\BNP\Jsons")
#os.mkdir(fecha)

fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\\BNPFijos.csv")
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BNP\\"+fecha+"\\DicisBNP")
docs=os.listdir() 
meses={"ene":"01","feb":"02","mar":"03","abr":"04","may":"05","jun":"06","jul":"07","ago":"08","sep":"09",
    "oct":"10","nov":"11","dic":"12"}
error=[]
for p in docs:
    try:
        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BNP\\"+fecha+"\\DicisBNP\\"+p
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BNP\\Templates\BNP_Template1.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BNP\\Templates\BNP_Template1Full.json"

        pdf = PyPDF2.PdfFileReader(rutapdf)

        #replicamos nuestro template para todas las series
        full_temp = list()
        for n_pags in range((pdf.getNumPages())//2):
            with open(rutatemplate) as f:
                    temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

                    #guardamos el nuevo template
        with open("BNP_Template1Full.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutafull)

        indexa=" ".join(data[1].T.reset_index().T.iloc[3:,2].dropna())
        razon=" ".join(data[0].T.reset_index().T.dropna().iloc[:,0])
        
        fech=" ".join(data[1].T.reset_index().T.iloc[3:,5].dropna())
        try:
            dia=fech.strip()[:2]
            mes=meses[fech.strip()[2:-2].strip().replace("-","")]
            año="20"+fech.strip()[-2:].strip()
            auxfech=año+mes+dia
        except:
            auxfech=None
        
        if "variable" in " ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()).lower() :
            tipoaux=101
        elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()).lower() :
            tipoaux=100 
           
        auxcal=" ".join(data[1].T.reset_index().T.iloc[3:,4].dropna())
        if("HR"in auxcal):
            auxcal=auxcal[auxcal.find("HR")+2:auxcal.find("/")].strip()+auxcal[auxcal.find("/"):auxcal.find("/")+3].replace(" ","")
        elif( "(mex)" in auxcal):
            auxcal=auxcal.replace("(mex)","").replace("f","").replace("S","")
            
        cierreaux=' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
        
        RES={
                          "fondo": {
                                "nombre":" ".join(data[1].T.reset_index().T.iloc[3:,2].dropna()),
                                "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                                "razon_social":{
                                    "valor":razon[razon.lower().find("la inver")+12:razon.lower().find("admin")].strip()
                                },
                                "fecha_autorizacion": {
                                    "valor":auxfech
                                },
                                "tipo": {
                                    "valor":" ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()),
                                    "id": tipoaux
                                },
                                "administradora": {
                                    "nombre":razon[razon.lower().find(" por ")+4:razon.lower().find(" por ")+4+razon.lower().find("de inver")+14].strip(),
                                    "valor": 85
                                },
                                "horizonte_inversion": {
                                    "valor": fijos.iloc[1,list(fijos.columns).index(indexa)].strip()
                                },
                                "liquidez": {
                                    "valor": ' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,0].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                },
                                "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[4].T.reset_index().T.iloc[:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                },
                                "calificacion_mercado": {
                                    "valor": auxcal.replace(" ","")
                                },
                                "director": {
                                    "valor": fijos.iloc[2,list(fijos.columns).index(indexa)].strip()
                                },
                                "vigente": 1,
                                "categoria": {
                                    "valor": " ".join(data[1].T.reset_index().T.iloc[3:,0].dropna())
                                },
                                "horario_cierre": {
                                    "valor":cierreaux,
                                },
                                "campos_faltantes": [],
                                "serie":[
                                ]
                            }
                        }

        aux=[]

        for i in range((pdf.getNumPages())//2):

            serie=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,3].dropna())
            adqui=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,6].dropna())
            limites=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
            comicliente=data[(6* i)+2].T.reset_index().iloc[:,0][1]
            comifondo=data[(6* i)+3].T.reset_index().iloc[:,0][1]

            aux1=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
            aux2=None
            try:
                if ( "máximo" in limites.lower() and "mínimo" in limites.lower()):
                    if("sin monto máximo" in limites.lower() and "y sin" not in limites.lower()):
                        aux1=limites[limites.lower().find("mínimo")+7:].strip().replace("Sin monto máximo","")
                        aux2=None
                    elif("y sin" in limites.lower()):
                        aux1=None
                        aux2=None
                    else:
                        aux1=limites[limites.lower().find("mínimo")+7:limites.lower().find("máximo")].strip()
                        aux2=limites[limites.lower().find("máximo")+7:].strip()
                elif(( "máximo" in limites.lower() and "mínimo" not in limites.lower())):
                    aux1=None
                    aux2=limites[limites.lower().find("máximo")+7:].strip()

                elif(( "máximo" not in limites.lower() and "mínimo" in limites.lower())):
                    if ( "y sin" in limites.lower()):
                        aux1=None
                        aux2=None
                    else:
                        aux1=limites[limites.lower().find("mínimo")+7:].strip()
                        aux2=None 
            except:
                aux1=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                aux2=None
                
            try:
                comicliente=comicliente.replace("%","")
            except:
                comicliente=comicliente

            try:
                comifondo=comifondo.replace("%","")
            except:
                comifondo=comifondo


            aux.append({"nombre": serie.strip().replace(".",""),
                                     "monto_minimo": {
                                         "valor": aux1
                                    },
                                     "monto_maximo": {
                                         "valor": aux2
                                    },
                                     "comision_cliente": {
                                         "valor": comicliente
                                    },
                                      "comision_fondo": {
                                         "valor": comifondo
                                    },
                                     "posibles_adquirientes": {
                                         "valor": adqui.strip().replace(".","")
                                    },
                                     "vigente":1
                                    ,
                                     "clave_BMV": RES["fondo"]["nombre"]+serie,
                                    "campos_faltantes": []
                                    })
        RES["fondo"]["serie"]=aux  

        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\Jsons\\"+fecha)
        with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"success {p}")
    except:
        try:
            direcactual=os.getcwd()
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\Templates")

            rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BNP\\"+fecha+"\\DicisBNP\\"+p
            rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BNP\\Templates\BNP_Template2.json"
            rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BNP\\Templates\BNP_Template1Full.json"

            pdf = PyPDF2.PdfFileReader(rutapdf)

            #replicamos nuestro template para todas las series
            full_temp = list()
            for n_pags in range((pdf.getNumPages())//2):
                with open(rutatemplate) as f:
                        temp = json.load(f)
                z = temp.copy()
                [x.update({"page":x["page"]+2*n_pags}) for x in z]
                full_temp.extend(z)

                        #guardamos el nuevo template
            with open("BNP_Template1Full.json", "w") as f:
                json.dump(full_temp,f)

            data = tabula.read_pdf_with_template(rutapdf,rutafull)

            indexa=" ".join(data[1].T.reset_index().T.iloc[3:,2].dropna())
            razon=" ".join(data[0].T.reset_index().T.dropna().iloc[:,0])
            
            fech=" ".join(data[1].T.reset_index().T.iloc[3:,5].dropna())
            try:
                dia=fech.strip()[:2]
                mes=meses[fech.strip()[2:-2].strip().replace("-","")]
                año="20"+fech.strip()[-2:].strip()
                auxfech=año+mes+dia
            except:
                auxfech=None

            if "variable" in " ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()).lower() :
                tipoaux=101
            elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()).lower() :
                tipoaux=100 

            auxcal=" ".join(data[1].T.reset_index().T.iloc[3:,4].dropna())
            if("HR"in auxcal):
                auxcal=auxcal[auxcal.find("HR")+2:auxcal.find("/")].strip()+auxcal[auxcal.find("/"):auxcal.find("/")+3].replace(" ","")
            elif( "(mex)" in auxcal):
                auxcal=auxcal.replace("(mex)","").replace("f","").replace("S","")

            cierreaux=' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
            cierreaux=cierreaux[cierreaux.find("a")+1:].strip()

            RES={
                              "fondo": {
                                    "nombre":" ".join(data[1].T.reset_index().T.iloc[3:,2].dropna()),
                                    "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                                    "razon_social":{
                                        "valor":razon[razon.lower().find("la inver")+12:razon.lower().find("admin")].strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":" ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find(" por ")+4:razon.lower().find(" por ")+4+razon.lower().find("de inver")+14].strip(),
                                        "valor": 85
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(indexa)].strip()
                                    },
                                    "liquidez": {
                                        "valor": ' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,0].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[4].T.reset_index().T.iloc[:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace(" ","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(indexa)].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[3:,0].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor":cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

            aux=[]

            for i in range((pdf.getNumPages())//2):

                serie=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,3].dropna())
                adqui=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,6].dropna())
                limites=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                comicliente=data[(6* i)+2].T.reset_index().iloc[:,0][1]
                comifondo=data[(6* i)+3].T.reset_index().iloc[:,0][1]

                aux1=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                aux2=None
                try:
                    if ( "máximo" in limites.lower() and "mínimo" in limites.lower()):
                        if("sin monto máximo" in limites.lower() and "y sin" not in limites.lower()):
                            aux1=limites[limites.lower().find("mínimo")+7:].strip().replace("Sin monto máximo","")
                            aux2=None
                        elif("y sin" in limites.lower()):
                            aux1=None
                            aux2=None
                        else:
                            aux1=limites[limites.lower().find("mínimo")+7:limites.lower().find("máximo")].strip()
                            aux2=limites[limites.lower().find("máximo")+7:].strip()
                    elif(( "máximo" in limites.lower() and "mínimo" not in limites.lower())):
                        aux1=None
                        aux2=limites[limites.lower().find("máximo")+7:].strip()

                    elif(( "máximo" not in limites.lower() and "mínimo" in limites.lower())):
                        if ( "y sin" in limites.lower()):
                            aux1=None
                            aux2=None
                        else:
                            aux1=limites[limites.lower().find("mínimo")+7:].strip()
                            aux2=None 
                except:
                    aux1=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                    aux2=None
                
                try:
                    comicliente=comicliente.replace("%","")
                except:
                    comicliente=comicliente

                try:
                    comifondo=comifondo.replace("%","")
                except:
                    comifondo=comifondo

                
                aux.append({"nombre": serie.strip().replace(".",""),
                                         "monto_minimo": {
                                             "valor": aux1
                                        },
                                         "monto_maximo": {
                                             "valor": aux2
                                        },
                                         "comision_cliente": {
                                             "valor": comicliente
                                        },
                                          "comision_fondo": {
                                             "valor": comifondo
                                        },
                                         "posibles_adquirientes": {
                                             "valor": adqui.strip().replace(".","")
                                        },
                                         "vigente":1
                                        ,
                                         "clave_BMV": RES["fondo"]["nombre"]+serie,
                                        "campos_faltantes": []
                                        })
            RES["fondo"]["serie"]=aux  

            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\Jsons\\"+fecha)
            with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
                json.dump(RES, f, indent=2,ensure_ascii=False)
            print(f"success {p}")
        except:
            try:
                direcactual=os.getcwd()
                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\Templates")

                rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BNP\\"+fecha+"\\DicisBNP\\"+p
                rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BNP\\Templates\BNP_Template3.json"
                rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BNP\\Templates\BNP_Template1Full.json"

                pdf = PyPDF2.PdfFileReader(rutapdf)

                #replicamos nuestro template para todas las series
                full_temp = list()
                for n_pags in range((pdf.getNumPages())//2):
                    with open(rutatemplate) as f:
                            temp = json.load(f)
                    z = temp.copy()
                    [x.update({"page":x["page"]+2*n_pags}) for x in z]
                    full_temp.extend(z)

                            #guardamos el nuevo template
                with open("BNP_Template1Full.json", "w") as f:
                    json.dump(full_temp,f)

                data = tabula.read_pdf_with_template(rutapdf,rutafull)

                indexa=" ".join(data[1].T.reset_index().T.iloc[3:,2].dropna())
                razon=" ".join(data[0].T.reset_index().T.dropna().iloc[:,0])
                
                
                fech=" ".join(data[1].T.reset_index().T.iloc[3:,5].dropna())
                try:
                    dia=fech.strip()[:2]
                    mes=meses[fech.strip()[2:-2].strip().replace("-","")]
                    año="20"+fech.strip()[-2:].strip()
                    auxfech=año+mes+dia
                except:
                    auxfech=None

                if "variable" in " ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()).lower() :
                    tipoaux=101
                elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()).lower() :
                    tipoaux=100 

                auxcal=" ".join(data[1].T.reset_index().T.iloc[3:,4].dropna())
                if("HR"in auxcal):
                    auxcal=auxcal[auxcal.find("HR")+2:auxcal.find("/")].strip()+auxcal[auxcal.find("/"):auxcal.find("/")+3].replace(" ","")
                elif( "(mex)" in auxcal):
                    auxcal=auxcal.replace("(mex)","").replace("f","").replace("S","")

                cierreaux=' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
                
                
                RES={
                                  "fondo": {
                                        "nombre":" ".join(data[1].T.reset_index().T.iloc[3:,2].dropna()),
                                        "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                                        "razon_social":{
                                            "valor":razon[razon.lower().find("la inver")+12:razon.lower().find("admin")].strip()
                                        },
                                        "fecha_autorizacion": {
                                            "valor":auxfech
                                        },
                                        "tipo": {
                                            "valor":" ".join(data[1].T.reset_index().T.iloc[3:,1].dropna()),
                                            "id": tipoaux
                                        },
                                        "administradora": {
                                            "nombre":razon[razon.lower().find(" por ")+4:razon.lower().find(" por ")+4+razon.lower().find("de inver")+14].strip(),
                                            "valor": 85
                                        },
                                        "horizonte_inversion": {
                                            "valor": fijos.iloc[1,list(fijos.columns).index(indexa)].strip()
                                        },
                                        "liquidez": {
                                            "valor": ' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,0].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                        },
                                        "permanencia": {
                                            "valor":' '.join( str(x) for x in list(data[4].T.reset_index().T.iloc[:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                        },
                                        "calificacion_mercado": {
                                            "valor": auxcal.replace(" ","")
                                        },
                                        "director": {
                                            "valor": fijos.iloc[2,list(fijos.columns).index(indexa)].strip()
                                        },
                                        "vigente": 1,
                                        "categoria": {
                                            "valor": " ".join(data[1].T.reset_index().T.iloc[3:,0].dropna())
                                        },
                                        "horario_cierre": {
                                            "valor":cierreaux,
                                        },
                                        "campos_faltantes": [],
                                        "serie":[
                                        ]
                                    }
                                }

                aux=[]

                for i in range((pdf.getNumPages())//2):

                    serie=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,3].dropna())
                    adqui=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,6].dropna())
                    limites=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                    comicliente=data[(6* i)+2].T.reset_index().iloc[:,0][1]
                    comifondo=data[(6* i)+3].T.reset_index().iloc[:,0][1]
                    
                    aux1=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                    aux2=None
                    try:
                        if ( "máximo" in limites.lower() and "mínimo" in limites.lower()):
                            if("sin monto máximo" in limites.lower() and "y sin" not in limites.lower()):
                                aux1=limites[limites.lower().find("mínimo")+7:].strip().replace("Sin monto máximo","")
                                aux2=None
                            elif("y sin" in limites.lower()):
                                aux1=None
                                aux2=None
                            else:
                                aux1=limites[limites.lower().find("mínimo")+7:limites.lower().find("máximo")].strip()
                                aux2=limites[limites.lower().find("máximo")+7:].strip()
                        elif(( "máximo" in limites.lower() and "mínimo" not in limites.lower())):
                            aux1=None
                            aux2=limites[limites.lower().find("máximo")+7:].strip()

                        elif(( "máximo" not in limites.lower() and "mínimo" in limites.lower())):
                            if ( "y sin" in limites.lower()):
                                aux1=None
                                aux2=None
                            else:
                                aux1=limites[limites.lower().find("mínimo")+7:].strip()
                                aux2=None 
                    except:
                        aux1=" ".join(data[(6*i)+1].T.reset_index().T.iloc[3:,7].dropna())
                        aux2=None
                    
                    try:
                        comicliente=comicliente.replace("%","")
                    except:
                        comicliente=comicliente
                        
                    try:
                        comifondo=comifondo.replace("%","")
                    except:
                        comifondo=comifondo



                    aux.append({"nombre": serie.strip().replace(".",""),
                                             "monto_minimo": {
                                                 "valor": aux1
                                            },
                                             "monto_maximo": {
                                                 "valor": aux2
                                            },
                                             "comision_cliente": {
                                                 "valor": comicliente
                                            },
                                              "comision_fondo": {
                                                 "valor": comifondo
                                            },
                                             "posibles_adquirientes": {
                                                 "valor": adqui.strip().replace(".","")
                                            },
                                             "vigente":1
                                            ,
                                             "clave_BMV": RES["fondo"]["nombre"]+serie,
                                            "campos_faltantes": []
                                            })
                RES["fondo"]["serie"]=aux  

                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BNP\Jsons\\"+fecha)
                with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
                    json.dump(RES, f, indent=2,ensure_ascii=False)
                print(f"success {p}")
            except Exception as e:
                print(e)
                print(f"error en {p}")
                error.append(p)

success AXAIM29.pdf
success AXAIM3A.pdf
success AXAIM41.pdf
success AXAIM53.pdf
success AXAIM65.pdf
success BNPP-50.pdf
success BNPP-60.pdf
success BNPP-70.pdf
success BNPP-90.pdf
success BNPP-CP.pdf
success BNPP-LP.pdf
success BNPP-RV.pdf
success HAYEK.pdf
success PORTMAN.pdf


### BBVA

In [2]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime 

fecha=str(datetime.date.today())
os.chdir("C:\\Users\\ortca\\Analysic_Nabla\\Proyectos\\INFOSEL\\ProcesoB\Fondos_root\BBVA\Jsons")
#os.mkdir(fecha)

fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\\BBVAFijos.csv")
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BBVA\\"+fecha+"\\DicisBBVA")
docs=os.listdir() 
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
    "octubre":"10","noviembre":"11","diciembre":"12"}

for p in docs:
    try:
        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BBVA\\"+fecha+"\\DicisBBVA\\"+p
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BBVA\\Templates\BBVA_Template1.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BBVA\\Templates\BBVA_Template1Full.json"

        pdf = PyPDF2.PdfFileReader(rutapdf)


        #replicamos nuestro template para todas las series
        full_temp = list()
        for n_pags in range((pdf.getNumPages())//2):
            with open(rutatemplate) as f:
                temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

            #guardamos el nuevo template
        with open("BBVA_Template1Full.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutafull)

        
        if "variable" in data[1].T.reset_index().iloc[:,0][1].lower() :
            tipoaux=101
        elif "deuda" in data[1].T.reset_index().iloc[:,0][1].lower() :
            tipoaux=100 
        
        
        if (len(data[0].columns)>1):
            indiceaux=1
        else:
            indiceaux=0
    
            
        a=' '.join( str(x) for x in list(data[0].T.reset_index().T.iloc[:,indiceaux]) if x != "Unnamed: 0" and type(x) != float)
        b=' '.join( str(x) for x in list(data[5].T.reset_index().T.iloc[1:,1]) if x != "Unnamed: 0" and type(x) != float)
        c=' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:,0]) if x != "Unnamed: 0" and type(x) != float)
        
        if "hasta" in b.lower():
            cierre=b[b.find("hasta")+5:]
        else:
            cierre=b[b.find("a")+2:]
        
        auxfecha=data[1].T.reset_index().T.iloc[:,1].dropna().tail(1).values[0].replace("/","") 
        if ( "calificadora" in auxfecha.lower()):
            auxfecha=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].T.reset_index().T.iloc[:,1].dropna())]
        
        auxcali=data[1].T.reset_index().iloc[:,3][1].strip()
        try:
            auxcali=auxcali.replace("Escala homogenea","").strip()
        except:
            auxcali=auxcali
        
        try:
            año=auxfecha[-4:]
            mes=auxfecha[2:-4] 
            dia=auxfecha[:2]
            fec=año+mes+dia
        except:
            fec=None
            
 
        indexa=data[1].T.reset_index().T.dropna().iloc[2][1]
        
        RES={
                  "fondo": {
                        "nombre":data[1].T.reset_index().T.dropna().iloc[2][1].strip().replace(".",""),
                        "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                        "razon_social":{
                            "valor": data[0].T.reset_index().iloc[:,0][indiceaux][:data[0].T.reset_index().iloc[:,0][indiceaux].find(", administrado")].strip()
                        },
                        "fecha_autorizacion": {
                            "valor":fec
                        },
                        "tipo": {
                            "valor":data[1].T.reset_index().iloc[:,0][1].strip().replace(".","") ,
                            "id": tipoaux
                        },
                        "administradora": {
                            "nombre":a[a.find("por")+3:].strip(),
                            "valor": 6
                        },
                        "horizonte_inversion": {
                            "valor": fijos.iloc[1,list(fijos.columns).index(indexa)].strip()
                        },
                        "liquidez": {
                            "valor": ' '.join( str(x) for x in list(data[5].T.reset_index().iloc[1:,0].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                        },
                        "permanencia": {
                            "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                        },
                        "calificacion_mercado": {
                            "valor": auxcali
                        },
                        "director": {
                            "valor": fijos.iloc[2,list(fijos.columns).index(indexa)].strip()
                        },
                        "vigente": 1,
                        "categoria": {
                            "valor": data[1].T.reset_index().iloc[:,1][1].strip().replace(".","")
                        },
                        "horario_cierre": {
                            "valor":cierre.strip().replace(".","") ,
                        },
                        "campos_faltantes": [],
                        "serie":[
                        ]
                    }
                }

        aux=[]
        for i in range((pdf.getNumPages())//2):
            #primero 1

            serie=" ".join(data[(7* i)+2].T.reset_index().dropna().T.iloc[1:,0])
            adqui=" ".join(data[(7* i)+2].T.reset_index().dropna().T.iloc[1:,1])
            limites=" ".join(data[(7* i)+2].T.reset_index().dropna().T.iloc[1:,2])
            comicliente=data[(7* i)+3].T.reset_index().iloc[:,0][1]
            comifondo=data[(7* i)+4].T.reset_index().iloc[:,0][1]
            
            if "empleados y jubilados" in adqui.lower():
                adqui="Empleados y jubilados del grupo financiero"
            elif "personas morales del" in adqui.lower():
                adqui="Personas Morales del Sistema Financiero Mex"
            elif "personas morales no" in adqui.lower():
                adqui="Personas morales no sujetas a retención "
            elif "personas físicas residentes" in adqui.lower():
                adqui="Personas Físicas residentes en el extranjero"
            elif "gestión portafolios morales" in adqui.lower() and "tnc" in serie.lower():
                adqui="Gestión Portafolios Morales No sujetas a retención"
            elif "personas físicas con" in adqui.lower():
                adqui="Personas Físicas con Actividad Empresarial"
            elif "personas morales residentes" in adqui.lower():
                adqui="Personas Morales residentes en el extranjero"
   
            
            
            aux.append({"nombre": serie.strip().replace(".",""),
                         "monto_minimo": {
                             "valor": limites.strip()
                        },
                         "monto_maximo": {
                             "valor": None
                        },
                         "comision_cliente": {
                             "valor": comicliente
                        },
                          "comision_fondo": {
                             "valor": comifondo
                        },
                         "posibles_adquirientes": {
                             "valor": adqui.strip().replace(".","")
                        },
                         "vigente":1
                        ,
                         "clave_BMV": RES["fondo"]["nombre"]+serie,
                        "campos_faltantes": []
                        })
        RES["fondo"]["serie"]=aux  
        
        


        # Guardamos el json final 
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\Jsons\\"+fecha)
        with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"success {p}")
    except :
        try:
            direcactual=os.getcwd()
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\Templates")

            rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BBVA\\"+fecha+"\\DicisBBVA\\"+p
            rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BBVA\\Templates\BBVA_Template2.json"
            rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BBVA\\Templates\BBVA_Template1Full.json"

            pdf = PyPDF2.PdfFileReader(rutapdf)


            #replicamos nuestro template para todas las series
            full_temp = list()
            for n_pags in range((pdf.getNumPages())//2):
                with open(rutatemplate) as f:
                    temp = json.load(f)
                z = temp.copy()
                [x.update({"page":x["page"]+2*n_pags}) for x in z]
                full_temp.extend(z)

                #guardamos el nuevo template
            with open("BBVA_Template1Full.json", "w") as f:
                json.dump(full_temp,f)

            data = tabula.read_pdf_with_template(rutapdf,rutafull)


            if "variable" in data[1].T.reset_index().iloc[:,0][1].lower() :
                tipoaux=101
            elif "deuda" in data[1].T.reset_index().iloc[:,0][1].lower() :
                tipoaux=100 


            if (len(data[0].columns)>1):
                indiceaux=1
            else:
                indiceaux=0


            a=' '.join( str(x) for x in list(data[0].T.reset_index().T.iloc[:,indiceaux]) if x != "Unnamed: 0" and type(x) != float)
            b=" ".join(data[5].T.reset_index().T.iloc[1:,1].dropna())
            c=' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:,0]) if x != "Unnamed: 0" and type(x) != float)

            if "hasta" in b.lower():
                cierre=b[b.find("hasta"):]
            else:
                cierre=b[b.find("a")+2:]

            auxfecha=data[1].T.reset_index().T.iloc[:,1].dropna().tail(1).values[0].replace("/","") 
            if ( "calificadora" in auxfecha.lower()):
                auxfecha=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].T.reset_index().T.iloc[:,1].dropna())]
            try:
                auxfecha=auxfecha.replace("Escala homogenea","")
            except:
                auxfecha=auxfecha
                
            auxcali=data[1].T.reset_index().iloc[:,3][1].strip()
            try:
                auxcali=auxcali.replace("Escala homogenea","").strip()
            except:
                auxcali=auxcali
        
                
            try:
                año=auxfecha[-4:]
                mes=auxfecha[2:-4] 
                dia=auxfecha[:2]
                fec=año+mes+dia
            except:
                fec=None


            indexa=data[1].T.reset_index().T.dropna().iloc[2][1]

            RES={
                      "fondo": {
                            "nombre":data[1].T.reset_index().T.dropna().iloc[2][1].strip().replace(".",""),
                            "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                            "razon_social":{
                                "valor": data[0].T.reset_index().iloc[:,0][indiceaux][:data[0].T.reset_index().iloc[:,0][indiceaux].find(", administrado")].strip()
                            },
                            "fecha_autorizacion": {
                                "valor":fec
                            },
                            "tipo": {
                                "valor":data[1].T.reset_index().iloc[:,0][1].strip().replace(".","") ,
                                "id": tipoaux
                            },
                            "administradora": {
                                "nombre":a[a.find("por")+3:].strip(),
                                "valor": 6
                            },
                            "horizonte_inversion": {
                                "valor": fijos.iloc[1,list(fijos.columns).index(indexa)].strip()
                            },
                            "liquidez": {
                                "valor": " ".join(data[5].T.reset_index().T.iloc[:1,1].dropna())
                            },
                            "permanencia": {
                                "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                            },
                            "calificacion_mercado": {
                                "valor": auxcali
                            },
                            "director": {
                                "valor":fijos.iloc[2,list(fijos.columns).index(indexa)].strip()
                            },
                            "vigente": 1,
                            "categoria": {
                                "valor": data[1].T.reset_index().iloc[:,1][1].strip().replace(".","")
                            },
                            "horario_cierre": {
                                "valor":cierre.strip().replace(".","") ,
                            },
                            "campos_faltantes": [],
                            "serie":[
                            ]
                        }
                    }

            aux=[]
            for i in range((pdf.getNumPages())//2):
                #primero 1

                serie=" ".join(data[(7* i)+2].T.reset_index().T.iloc[1:,0].dropna())
                adqui=" ".join(data[(7* i)+2].T.reset_index().T.iloc[1:,1].dropna())
                limites=" ".join(data[(7* i)+2].T.reset_index().T.iloc[1:,2].dropna())
                comicliente=data[(7* i)+3].T.reset_index().iloc[:,0][1]
                comifondo=data[(7* i)+4].T.reset_index().iloc[:,0][1]

                if "empleados y jubilados" in adqui.lower():
                    adqui="Empleados y jubilados del grupo financiero"
                elif "personas morales del" in adqui.lower():
                    adqui="Personas Morales del Sistema Financiero Mex"
                elif "personas morales no" in adqui.lower():
                    adqui="Personas morales no sujetas a retención "
                elif "personas físicas residentes" in adqui.lower():
                    adqui="Personas Físicas residentes en el extranjero"
                elif "gestión portafolios morales" in adqui.lower() and "tnc" in serie.lower():
                    adqui="Gestión Portafolios Morales No sujetas a retención"
                elif "personas físicas con" in adqui.lower():
                    adqui="Personas Físicas con Actividad Empresarial"
                elif "personas morales residentes" in adqui.lower():
                    adqui="Personas Morales residentes en el extranjero"



                aux.append({"nombre": serie.strip().replace(".",""),
                             "monto_minimo": {
                                 "valor": limites.strip()
                            },
                             "monto_maximo": {
                                 "valor": None
                            },
                             "comision_cliente": {
                                 "valor": comicliente
                            },
                              "comision_fondo": {
                                 "valor": comifondo
                            },
                             "posibles_adquirientes": {
                                 "valor": adqui.strip().replace(".","")
                            },
                             "vigente":1
                            ,
                             "clave_BMV": RES["fondo"]["nombre"]+serie,
                            "campos_faltantes": []
                            })
            RES["fondo"]["serie"]=aux  




            # Guardamos el json final 
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\Jsons\\"+fecha)
            with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
                json.dump(RES, f, indent=2,ensure_ascii=False)
            print(f"success {p}")
        except:
            try:
                direcactual=os.getcwd()
                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\Templates")

                rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\BBVA\\"+fecha+"\\DicisBBVA\\"+p
                rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BBVA\\Templates\BBVA_Template3.json"
                rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\BBVA\\Templates\BBVA_Template1Full.json"

                pdf = PyPDF2.PdfFileReader(rutapdf)


                #replicamos nuestro template para todas las series
                full_temp = list()
                for n_pags in range((pdf.getNumPages())//2):
                    with open(rutatemplate) as f:
                        temp = json.load(f)
                    z = temp.copy()
                    [x.update({"page":x["page"]+2*n_pags}) for x in z]
                    full_temp.extend(z)

                    #guardamos el nuevo template
                with open("BBVA_Template1Full.json", "w") as f:
                    json.dump(full_temp,f)

                data = tabula.read_pdf_with_template(rutapdf,rutafull)


                if "variable" in data[1].T.reset_index().iloc[:,0][1].lower() :
                    tipoaux=101
                elif "deuda" in data[1].T.reset_index().iloc[:,0][1].lower() :
                    tipoaux=100 


                if (len(data[0].columns)>1):
                    indiceaux=1
                else:
                    indiceaux=0


                a=' '.join( str(x) for x in list(data[0].T.reset_index().T.iloc[:,indiceaux]) if x != "Unnamed: 0" and type(x) != float)
                b=" ".join(data[5].T.reset_index().T.iloc[1:,1].dropna())
                c=' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:,0]) if x != "Unnamed: 0" and type(x) != float)

                if "hasta" in b.lower():
                    cierre=b[b.find("hasta"):]
                else:
                    cierre=b[b.find("a")+2:]

                auxfecha=data[1].T.reset_index().T.iloc[:,1].dropna().tail(1).values[0].replace("/","") 
                if ( "calificadora" in auxfecha.lower()):
                    auxfecha=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].T.reset_index().T.iloc[:,1].dropna())]
                try:
                    auxfecha=auxfecha.replace("Escala homogenea","")
                except:
                    auxfecha=auxfecha
                    
                auxcali=data[1].T.reset_index().iloc[:,3][1].strip()
                try:
                    auxcali=auxcali.replace("Escala homogenea","").strip()
                except:
                    auxcali=auxcali

                try:
                    año=auxfecha[-4:]
                    mes=auxfecha[2:-4] 
                    dia=auxfecha[:2]
                    fec=año+mes+dia
                except:
                    fec=None


                indexa=data[1].T.reset_index().T.dropna().iloc[2][1]

                RES={
                          "fondo": {
                                "nombre":data[1].T.reset_index().T.dropna().iloc[2][1].strip().replace(".",""),
                                "url":fijos.iloc[0,list(fijos.columns).index(indexa)],
                                "razon_social":{
                                    "valor": data[0].T.reset_index().iloc[:,0][indiceaux][:data[0].T.reset_index().iloc[:,0][indiceaux].find(", administrado")].strip()
                                },
                                "fecha_autorizacion": {
                                    "valor":fec
                                },
                                "tipo": {
                                    "valor":data[1].T.reset_index().iloc[:,0][1].strip().replace(".","") ,
                                    "id": tipoaux
                                },
                                "administradora": {
                                    "nombre":a[a.find("por")+3:].strip(),
                                    "valor": 6
                                },
                                "horizonte_inversion": {
                                    "valor": fijos.iloc[1,list(fijos.columns).index(indexa)].strip()
                                },
                                "liquidez": {
                                    "valor": " ".join(data[5].T.reset_index().T.iloc[:1,1].dropna())
                                },
                                "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip().replace(".","")
                                },
                                "calificacion_mercado": {
                                    "valor": auxcali
                                },
                                "director": {
                                    "valor":fijos.iloc[2,list(fijos.columns).index(indexa)].strip()
                                },
                                "vigente": 1,
                                "categoria": {
                                    "valor": data[1].T.reset_index().iloc[:,1][1].strip().replace(".","")
                                },
                                "horario_cierre": {
                                    "valor":cierre.strip().replace(".","") ,
                                },
                                "campos_faltantes": [],
                                "serie":[
                                ]
                            }
                        }

                aux=[]
                for i in range((pdf.getNumPages())//2):
                    #primero 1

                    serie=" ".join(data[(7* i)+2].T.reset_index().T.iloc[1:,0].dropna())
                    adqui=" ".join(data[(7* i)+2].T.reset_index().T.iloc[1:,1].dropna())
                    limites=" ".join(data[(7* i)+2].T.reset_index().T.iloc[1:,2].dropna())
                    comicliente=data[(7* i)+3].T.reset_index().iloc[:,0][1]
                    comifondo=data[(7* i)+4].T.reset_index().iloc[:,0][1]

                    if "empleados y jubilados" in adqui.lower():
                        adqui="Empleados y jubilados del grupo financiero"
                    elif "personas morales del" in adqui.lower():
                        adqui="Personas Morales del Sistema Financiero Mex"
                    elif "personas morales no" in adqui.lower():
                        adqui="Personas morales no sujetas a retención "
                    elif "personas físicas residentes" in adqui.lower():
                        adqui="Personas Físicas residentes en el extranjero"
                    elif "gestión portafolios morales" in adqui.lower() and "tnc" in serie.lower():
                        adqui="Gestión Portafolios Morales No sujetas a retención"
                    elif "personas físicas con" in adqui.lower():
                        adqui="Personas Físicas con Actividad Empresarial"
                    elif "personas morales residentes" in adqui.lower():
                        adqui="Personas Morales residentes en el extranjero"



                    aux.append({"nombre": serie.strip().replace(".",""),
                                 "monto_minimo": {
                                     "valor": limites.strip()
                                },
                                 "monto_maximo": {
                                     "valor": None
                                },
                                 "comision_cliente": {
                                     "valor": comicliente
                                },
                                  "comision_fondo": {
                                     "valor": comifondo
                                },
                                 "posibles_adquirientes": {
                                     "valor": adqui.strip().replace(".","")
                                },
                                 "vigente":1
                                ,
                                 "clave_BMV": RES["fondo"]["nombre"]+serie,
                                "campos_faltantes": []
                                })
                RES["fondo"]["serie"]=aux  




                # Guardamos el json final 
                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\BBVA\Jsons\\"+fecha)
                with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
                    json.dump(RES, f, indent=2,ensure_ascii=False)
                print(f"success {p}")
            except Exception as e:
                print(f"error enfondo {p}")
                print(e)
                error.append(p)

                



success B+EDUCA.pdf
success B+EST6.pdf
success B+RVUS2.pdf
success BBVAA.pdf
success BBVAAD.pdf
success BBVAAF.pdf
success BBVAAP.pdf
success BBVAC+.pdf
success BBVAC.pdf
success BBVACAP.pdf
success BBVACD+.pdf
success BBVACD.pdf
success BBVACOR.pdf
success BBVACR.pdf
success BBVACRD.pdf
success BBVACRE.pdf
success BBVADLP.pdf
success BBVADOL.pdf
success BBVAE50.pdf
success BBVAEMG.pdf
success BBVAESG.pdf
success BBVAEUR.pdf
success BBVAGB2.pdf
success BBVAGBP.pdf
success BBVAGF.pdf
success BBVAGFD.pdf
success BBVAGL3.pdf
success BBVAGL4.pdf
success BBVAGL5.pdf
success BBVAGLP.pdf
success BBVAGOB.pdf
success BBVAGUB.pdf
success BBVALIQ.pdf
success BBVALQ.pdf
success BBVAM.pdf
success BBVAMD.pdf
success BBVAMLT.pdf
success BBVAMRF.pdf
success BBVANDQ.pdf
success BBVAPC.pdf
success BBVAPCD.pdf
success BBVAPYM.pdf
success BBVAPZO.pdf
success BBVARF.pdf
success BBVARV.pdf
success BBVARV2.pdf
success BBVASIC.pdf
success BBVATES.pdf
success BBVAUDI.pdf
success BBVAUMS.pdf
success BBVAUS.pdf


### Actinver

In [34]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime 

fecha=str(datetime.date.today())
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Jsons")
#os.mkdir(fecha)

fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\\\ActinverFijos.csv")
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\Actinver\\"+fecha+"\\DicisActinver")
docs=os.listdir()
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
    "octubre":"10","noviembre":"11","diciembre":"12"}
error=[]

for a in docs:
    try:
        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\Actinver\\"+fecha+"\\DicisActinver\\" + a
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates\ACTINVER_Template1.json"
        rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates\ACTINVER_Template2.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates\ACTINVER_TemplateFull.json"
        pdf = PyPDF2.PdfFileReader(rutapdf)


        #replicamos nuestro template para todas las series
        full_temp = list()
        for n_pags in range((pdf.getNumPages()-2)//2):
            with open(rutaaux) as f:     
                temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

        #guardamos el nuevo template
        with open("ACTINVER_TemplateFull.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
        data2 = tabula.read_pdf_with_template(rutapdf,rutafull)


        if "variable" in data[0].T.reset_index().iloc[:,0][1].lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().iloc[:,0][1].lower():
            tipoaux=100 
        fech=data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())]

        try:
            dia=fech.strip()[:2]
            mes=meses[fech.strip()[2:-4].strip()]
            año=fech.strip()[-4:].strip()
            auxfech=año+mes+dia
        except:
            auxfech=None

        if (a== "APOLO 10.pdf"):
            indaux="APOLO 10"
        elif (a== "REGIO 1.pdf"):
            indaux="REGIO 1"
        elif (a== "+VALOR.pdf"):
            indaux="MASVALOR"
        else:
            indaux=data[0].T.reset_index().iloc[:,2][1].strip().replace(".","")
            
        try:    
            cierre=" ".join(data[4].T.reset_index().T.iloc[1:,1])[" ".join(data[4].T.reset_index().T.iloc[1:,1]).lower().find("cierre")+7:].strip()
        except:
            cierre=" ".join(data[4].T.reset_index().T.iloc[1:,1])
        try:
            calif=data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())-1][:data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())-1].find("/")]+data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())-1][data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())-1].find("/"):data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())-1].find("/")+2]
        except:
            calif=data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())-1]
        
        
        RES={
                      "fondo": {
                            "nombre": data[0].T.reset_index().iloc[:,2][1].strip().replace(".",""),
                            "url": fijos.iloc[0,list(fijos.columns).index(indaux)].strip(),
                            "razon_social": {
                                "valor": " ".join((data[6].T.reset_index().T.iloc[0:2,1:]).iloc[0,:].dropna() ).replace("Unnamed: 0","").replace(".1","").replace(".2","")+" ".join((data[6].T.reset_index().T.iloc[0:2,1:]).iloc[1,:].dropna())
                            },
                            "fecha_autorizacion": {
                                "valor": auxfech
                            },
                            "tipo": {
                                "valor": data[0].T.reset_index().iloc[:,0][1].strip().replace(".",""),
                                "id": tipoaux
                            },
                            "administradora": {
                                "nombre":"Operadora Actinver, S.A. de C.V., Sociedad Operadora de Fondos de Inversión,Grupo Financiero Actinver (ACTINVER)",
                                "valor": 2
                            },
                            "horizonte_inversion": {
                                "valor": fijos.iloc[1,list(fijos.columns).index(indaux)].strip()
                            },
                            "liquidez": {
                                "valor":  data[4].T.reset_index().iloc[:,0][1].strip().replace(".","")
                            },
                            "permanencia": {
                                "valor": data[5].T.reset_index().iloc[:,0][1].strip().replace(".","")
                            },
                            "calificacion_mercado": {
                                "valor":calif
                            },
                            "director": {
                                "valor": fijos.iloc[2,list(fijos.columns).index(indaux)].strip()
                            },
                            "vigente": 1,
                            "categoria": {
                                "valor": data[0].T.reset_index().iloc[:,1][1].strip().replace(".","").replace("\r","")
                            },
                            "horario_cierre": {
                                "valor": cierre,
                            },
                            "campos_faltantes": [],
                            "serie":[

                            ]
                        }
                    }
        aux=[]
        if ( len(data[1].columns)==3):
            limaux=" ".join(data[1].T.reset_index().T.iloc[1:,2].dropna())
            limaux2=None
            #primero 1 
            serie=" ".join(data[1].T.reset_index().T.iloc[1:,0].dropna())
            adqui=" ".join(data[1].T.reset_index().T.iloc[1:,1].dropna())
            limites=" ".join(data[1].T.reset_index().T.iloc[1:,2].dropna())
            comicliente=data[2].T.reset_index().iloc[:,0][1]
            comifondo=data[3].T.reset_index().iloc[:,0][1]

            try:    
                if(("–" in limites) and ("adelante" not in limites) ):
                    limaux=limites[:limites.find("–")].strip()
                    limaux2=limites[limites.find("–")+1:].strip()
                elif(("-" in limites) and ("adelante" not in limites)):
                    limaux=limites[:limites.find("-")].strip()
                    limaux2=limites[limites.find("-")+1:].strip()
                elif(("–" in limites) and ("adelante"  in limites) ):
                    limaux=limites[:limites.find("–")].strip()
                    limaux2=None
                elif(("-" in limites) and ("adelante" in limites)):
                    limaux=limites[:limites.find("-")].strip()
                    limaux2=None
                elif(("en adelante" in limites)):
                    limaux=limites[:limites.find("en adelante")].strip()
                    limaux2=None
                else:
                    limaux=limites.strip()
            except:
                limaux=limites.strip()
                limaux2=None


            if ("-" in serie[serie.find('“')+1:]):
                auxserie=serie[serie.find('Clase')+5:]
            else:
                auxserie=serie[serie.find('Clase')+5:]


        aux.append({   "nombre": auxserie.replace('"',"").replace('“',"").strip().replace(".",""),
                                    "monto_minimo": {
                                    "valor": limaux
                                    },
                                    "monto_maximo": {
                                    "valor": limaux2
                                    },
                                    "comision_cliente": {
                                    "valor": comicliente
                                    },
                                    "comision_fondo": {
                                    "valor": comifondo
                                     },
                                    "posibles_adquirientes": {
                                    "valor": adqui.strip().replace(".","")
                                    },
                                    "vigente": 1
                                    ,
                                    "clave_BMV": RES["fondo"]["nombre"]+auxserie.replace('"',"").replace('“',"").strip(),
                                    "campos_faltantes": []
                                })

        for i in range((pdf.getNumPages()-2)//2):  
            if ( len(data[1].columns)==3):
                limaux=" ".join(data2[3*i].T.reset_index().T.iloc[1:,2].dropna())
                limaux2=None

                serie=" ".join(data2[3*i].T.reset_index().T.iloc[1:,0].dropna())
                adqui=" ".join(data2[3*i].T.reset_index().T.iloc[1:,1].dropna())
                limites=" ".join(data2[3*i].T.reset_index().T.iloc[1:,2].dropna())
                comicliente=data2[3*(i)+1].T.reset_index().iloc[:,0][1]
                comifondo=data2[3*(i)+2].T.reset_index().iloc[:,0][1]

                try:    
                    if(("–" in limites) and ("adelante" not in limites) ):
                        limaux=limites[:limites.find("–")].strip()
                        limaux2=limites[limites.find("–")+1:].strip()
                    elif(("-" in limites) and ("adelante" not in limites)):
                        limaux=limites[:limites.find("-")].strip()
                        limaux2=limites[limites.find("-")+1:].strip()
                    elif(("–" in limites) and ("adelante"  in limites) ):
                        limaux=limites[:limites.find("–")].strip()
                        limaux2=None
                    elif(("-" in limites) and ("adelante" in limites)):
                        limaux=limites[:limites.find("-")].strip()
                        limaux2=None
                    elif(("en adelante" in limites)):
                        limaux=limites[:limites.find("en adelante")].strip()
                        limaux2=None
                    else:
                        limaux=limites.strip()
                except:
                    limaux=limites.strip()
                    limaux2=None

                if ("-" in serie[serie.find('“')+1:]):
                    auxserie=serie[serie.find('Clase')+5:]
                else:
                    auxserie=serie[serie.find('Clase')+5:]

                if "EMPLEADOS DE GRUPO" in adqui:
                    adqui="EMPLEADOS DE GRUPO FINANCIERO ACTINVER"
                elif "PERSONAS NO SUJETAS A" in adqui:
                    adqui="PERSONAS NO SUJETAS A RETENCIÓN"
                elif "PERSONAS MORALES NO" in adqui and "CE" in auxserie :
                    adqui="PERSONAS MORALES NO CONTRIBUYENTES CON GESTIÓN DE INVERSIONES"
                elif "GESTIÓN DE INVERSIÓN" in adqui:
                    adqui="GESTIÓN DE INVERSIÓN ADMINISTRADA POR LA SOCIEDAD OPERADORA"

            aux.append({   "nombre": auxserie.replace('"',"").replace('“',"").strip().replace(".",""),
                                    "monto_minimo": {
                                    "valor": limaux
                                    },
                                    "monto_maximo": {
                                    "valor": limaux2
                                    },
                                    "comision_cliente": {
                                    "valor": comicliente
                                    },
                                    "comision_fondo": {
                                    "valor": comifondo
                                     },
                                    "posibles_adquirientes": {
                                    "valor": adqui.strip().replace(".","")
                                    },
                                    "vigente":  1
                                    ,
                                    "clave_BMV": RES["fondo"]["nombre"]+auxserie.replace('"',"").replace('“',"").strip(),
                                    "campos_faltantes": []
                                })

        RES["fondo"]["serie"]=aux 

        # Guardamos el json final 
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Jsons\\"+fecha)

        with open(fecha.replace("-","")+"_"+a[:a.find(".")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"Success fondo: {a}")
    except:
        try:
            direcactual=os.getcwd()
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates")

            rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\Actinver\\"+fecha+"\\DicisActinver\\" + a
            rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates\ACTINVER_Template3.json"
            rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Templates\ACTINVER_TemplateFull.json"
            pdf = PyPDF2.PdfFileReader(rutapdf)


            #replicamos nuestro template para todas las series
            full_temp = list()
            for n_pags in range((pdf.getNumPages())//2):
                with open(rutatemplate) as f:     
                    temp = json.load(f)
                z = temp.copy()
                [x.update({"page":x["page"]+2*n_pags}) for x in z]
                full_temp.extend(z)

            #guardamos el nuevo template
            with open("ACTINVER_TemplateFull.json", "w") as f:
                json.dump(full_temp,f)

            data = tabula.read_pdf_with_template(rutapdf,rutafull)



            if "variable" in data[0].T.reset_index().iloc[:,0][1].lower():
                tipoaux=101
            elif "deuda" in data[0].T.reset_index().iloc[:,0][1].lower():
                tipoaux=100 
            fech=data[0].T.reset_index().T.iloc[:,1].dropna().values[4]

            try:
                dia=fech.strip()[:2]
                mes=meses[fech.strip()[2:-4].replace("del","").replace("de","").strip()]
                año=fech.strip()[-4:].strip()
                auxfech=año+mes+dia
            except:
                auxfech=None

            if (a== "APOLO 10.pdf"):
                indaux="APOLO 10"
            elif (a== "REGIO 1.pdf"):
                indaux="REGIO 1"
            elif (a== "+VALOR.pdf"):
                indaux="MASVALOR"
            else:
                indaux=data[0].T.reset_index().iloc[:,2][1].strip().replace(".","")
            
            try:
                cierre=" ".join(data[4].T.reset_index().T.iloc[1:,1])[" ".join(data[4].T.reset_index().T.iloc[1:,1]).lower().find("cierre")+7:].strip()
            except:
                cierre=" ".join(data[4].T.reset_index().T.iloc[1:,1])
            
            
            
            auxcal=data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())]
            try:
                calif=auxcal[:auxcal.find("/")]+auxcal[auxcal.find("/"):auxcal.find("/")+2]
            except:
                calif=auxcal
            
            RES={
                          "fondo": {
                                "nombre": data[0].T.reset_index().iloc[:,2][1].strip().replace(".",""),
                                "url": fijos.iloc[0,list(fijos.columns).index(indaux)].strip(),
                                "razon_social": {
                                    "valor": " ".join((data[6].T.reset_index().T.iloc[0:2,1:]).iloc[0,:].dropna() ).replace("Unnamed: 0","").replace(".1","").replace(".2","")+" ".join((data[6].T.reset_index().T.iloc[0:2,1:]).iloc[1,:].dropna())
                                },
                                "fecha_autorizacion": {
                                    "valor": auxfech
                                },
                                "tipo": {
                                    "valor": data[0].T.reset_index().iloc[:,0][1].strip().replace(".",""),
                                    "id": tipoaux
                                },
                                "administradora": {
                                    "nombre":"Operadora Actinver, S.A. de C.V., Sociedad Operadora de Fondos de Inversión,Grupo Financiero Actinver (ACTINVER)",
                                    "valor": 2
                                },
                                "horizonte_inversion": {
                                    "valor": fijos.iloc[1,list(fijos.columns).index(indaux)].strip()
                                },
                                "liquidez": {
                                    "valor":  data[4].T.reset_index().iloc[:,0][1].strip().replace(".","")
                                },
                                "permanencia": {
                                    "valor": data[5].T.reset_index().iloc[:,0][1].strip().replace(".","")
                                },
                                "calificacion_mercado": {
                                    "valor":data[0].T.reset_index().T.iloc[:,1].dropna().values[len(data[0].dropna())]
                                },
                                "director": {
                                    "valor": fijos.iloc[2,list(fijos.columns).index(indaux)].strip()
                                },
                                "vigente": 1,
                                "categoria": {
                                    "valor":data[0].T.reset_index().iloc[:,1][1].strip().replace(".","").replace("\r","")
                                },
                                "horario_cierre": {
                                    "valor": cierre,
                                },
                                "campos_faltantes": [],
                                "serie":[

                                ]
                            }
                        }
            print("entra2")
            for i in range((pdf.getNumPages()-2)//2):      
                limaux= " ".join(data[(7*i)+1].T.reset_index().T.iloc[1:,2].dropna())
                limaux2=None

                serie=" ".join(data[(7*i)+1].T.reset_index().T.iloc[1:,0].dropna())
                adqui=" ".join(data[(7*i)+1].T.reset_index().T.iloc[1:,1].dropna())
                limites= " ".join(data[(7*i)+1].T.reset_index().T.iloc[1:,2].dropna())
                comicliente=data[(7*i)+2].T.reset_index().iloc[:,0][1]
                comifondo=data[(7*i)+3].T.reset_index().iloc[:,0][1]

                try:    
                    if(("–" in limites) and ("adelante" not in limites) ):
                        limaux=limites[:limites.find("–")].strip()
                        limaux2=limites[limites.find("–")+1:].strip()
                    elif(("-" in limites) and ("adelante" not in limites)):
                        limaux=limites[:limites.find("-")].strip()
                        limaux2=limites[limites.find("-")+1:].strip()
                    elif(("–" in limites) and ("adelante"  in limites) ):
                        limaux=limites[:limites.find("–")].strip()
                        limaux2=None
                    elif(("-" in limites) and ("adelante" in limites)):
                        limaux=limites[:limites.find("-")].strip()
                        limaux2=None
                    elif(("en adelante" in limites)):
                        limaux=limites[:limites.find("en adelante")].strip()
                        limaux2=None
                    else:
                        limaux=limites.strip()
                except:
                    limaux=limites.strip()
                    limaux2=None

                if ("-" in serie[serie.find('“')+1:]):
                    auxserie=serie[serie.find('Clase')+5:]
                else:
                    auxserie=serie[serie.find('Clase')+5:]

                if "EMPLEADOS DE GRUPO" in adqui:
                    adqui="EMPLEADOS DE GRUPO FINANCIERO ACTINVER"
                elif "PERSONAS NO SUJETAS A" in adqui:
                    adqui="PERSONAS NO SUJETAS A RETENCIÓN"
                elif "PERSONAS MORALES NO" in adqui and "CE" in auxserie :
                    adqui="PERSONAS MORALES NO CONTRIBUYENTES CON GESTIÓN DE INVERSIONES"
                elif "GESTIÓN DE INVERSIÓN" in adqui:
                    adqui="GESTIÓN DE INVERSIÓN ADMINISTRADA POR LA SOCIEDAD OPERADORA"    

                aux.append({   "nombre": auxserie.replace('"',"").replace('“',"").strip().replace(".",""),
                                        "monto_minimo": {
                                        "valor": limaux
                                        },
                                        "monto_maximo": {
                                        "valor": limaux2
                                        },
                                        "comision_cliente": {
                                        "valor": comicliente
                                        },
                                        "comision_fondo": {
                                        "valor": comifondo
                                         },
                                        "posibles_adquirientes": {
                                        "valor": adqui.strip().replace(".","")
                                        },
                                        "vigente":  1
                                        ,
                                        "clave_BMV": RES["fondo"]["nombre"]+auxserie.replace('"',"").replace('“',"").strip(),
                                        "campos_faltantes": []
                                    })

            RES["fondo"]["serie"]=aux 
            # Guardamos el json final 
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\Actinver\Jsons\\"+fecha)

            with open(fecha.replace("-","")+"_"+a[:a.find(".")]+".json", "w", encoding="ansi") as f:
                json.dump(RES, f, indent=2,ensure_ascii=False)
            print(f"Success fondo: {a}")
        except Exception as e:
            print(f"error en fondo {a}")
            print(e)
            error.append(a)


Success fondo: +VALOR.pdf
Success fondo: ACT2025.pdf
Success fondo: ACT2030.pdf
Success fondo: ACT2035.pdf
Success fondo: ACT2040.pdf
Success fondo: ACT4560.pdf
Success fondo: ACTDUAL.pdf
Success fondo: ACTI500.pdf
Success fondo: ACTICOB.pdf
Success fondo: ACTICRE.pdf
Success fondo: ACTIEUR.pdf
Success fondo: ACTIG+.pdf
Success fondo: ACTIG+2.pdf
Success fondo: ACTIGOB.pdf
Success fondo: ACTIMED.pdf
Success fondo: ACTINMO.pdf
Success fondo: ACTINTK.pdf
entra2
Success fondo: ACTIOP1.pdf
Success fondo: ACTIPLU.pdf
Success fondo: ACTIPT1.pdf
Success fondo: ACTIPT4.pdf
Success fondo: ACTIREN.pdf
Success fondo: ACTIVAR.pdf
Success fondo: ACTOTAL.pdf
Success fondo: ACTPT13.pdf
Success fondo: ACTVIDA.pdf
Success fondo: AGOB+.pdf
Success fondo: ALTERN.pdf
Success fondo: APOLO 10.pdf
Success fondo: DIGITAL.pdf
Success fondo: DINAMO.pdf
entra2
Success fondo: ECOFUND.pdf
Success fondo: ESCALA.pdf
Success fondo: ESFERA.pdf
Success fondo: EVEREST.pdf
Success fondo: IMPULSA.pdf
Success fondo: MAXIMO

In [17]:
serie=" ".join(data[1].T.reset_index().T.iloc[1:,0].dropna())
        adqui=" ".join(data[1].T.reset_index().T.iloc[1:,1].dropna())
        limites=" ".join(data[1].T.reset_index().T.iloc[1:,2].dropna())

'PERSONAS FÍSICAS'

In [24]:
" ".join(data[1].T.reset_index().T.iloc[1:,1].dropna())

'Sin montos mínimos'

In [28]:
len(data[1].columns)

3

### SCOTIA

In [3]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime 


fecha=str(datetime.date.today())
os.chdir("C:\\Users\\ortca\\Analysic_Nabla\\Proyectos\\INFOSEL\\ProcesoB\\Fondos_root\\SCOTIA\\Jsons")
#os.mkdir(fecha)


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\\SCOTIAFijos.csv")
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SCOTIA\\"+fecha+"\\DicisSCOTIA")
docs=os.listdir()
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
    "octubre":"10","noviembre":"11","diciembre":"12"}
error=[]

for p in docs:
    try:

        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SCOTIA\\"+fecha+"\\DicisSCOTIA\\"+p
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_Template1.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_Template1Full.json"

        pdf = PyPDF2.PdfFileReader(rutapdf)


        full_temp = list()
        for n_pags in range((pdf.getNumPages())//2):
            with open(rutatemplate) as f:
                temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

                    #guardamos el nuevo template
        with open("SCOTIA_Template1Full.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutafull)
        
        if "variable" in data[1].T.reset_index().T.dropna().iloc[0][1].lower():
            tipoaux=101
        elif "deuda" in data[1].T.reset_index().T.dropna().iloc[0][1].lower():
            tipoaux=100 
        
        
        fech=data[2].T.reset_index().T.dropna().iloc[0][1].strip()
        
        try:
            dia=fech.strip()[:2]
            mes=meses[fech.strip()[2:-4].strip()]
            año=fech.strip()[-4:].strip()
            auxfech=año+mes+dia
        except:
            auxfech=None
        
        auxhora=" ".join(data[6].T.reset_index().T.dropna().iloc[1:,1])
        
        if("-" in auxhora or (("hasta" in auxhora.lower()) and ("-" not in auxhora.lower()))):
            auxhora=auxhora[auxhora.lower().find("hasta"):][auxhora[auxhora.lower().find("hasta"):].find(":")-3:].strip()
        elif("hasta" not in auxhora.lower()):
            auxhora=auxhora[auxhora.find(" a ")+2:].strip().replace(" a ","")

        
        
        razon=" ".join(data[0].T.reset_index().T.iloc[:,0])
        
        permaux=' '.join( str(x) for x in list(data[5].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
        
        if(("no tiene" in permaux.lower()) or ("no se tiene" in permaux.lower()) ):
            permaux="No se tiene plazo mínimo de permanencia"
            
        auxcalif=data[1].T.reset_index().T.dropna().iloc[len(data[1].dropna())][1].strip()    
        
        if(len(auxcalif)<15):
            auxcalif=auxcalif[:auxcalif.find("/")].replace("f","").replace("F","")+auxcalif[auxcalif.find("/"):].replace("s","").replace("S","").replace("(mex)","")
        else:
            auxcalif=auxcalif[:auxcalif.find("/")].replace("f","").replace("F","")+auxcalif[auxcalif.find("/"):auxcalif.find("/")+8].replace("s","").replace("S","").replace("(mex)","")
   
        
        RES={
          "fondo": {
                "nombre": data[1].T.reset_index().T.dropna().iloc[2][1].strip(),
                "url": fijos.iloc[0,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                "razon_social": {
                    "valor": razon[:razon.lower().find("admin")].strip()
                },
                "fecha_autorizacion": {
                    "valor": auxfech
                },
                "tipo": {
                    "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip(),
                    "id": tipoaux
                },
                "administradora": {
                    "nombre":razon[razon.lower().find("admin")+16:].strip(),
                    "valor": 8
                },
                "horizonte_inversion": {
                    "valor":  fijos.iloc[1,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                },
                "liquidez": {
                    "valor": data[6].T.reset_index().T.dropna().iloc[0][1].strip()
                },
                "permanencia": {
                    "valor":permaux
                },
                "calificacion_mercado": {
                    "valor": auxcalif
                },
                "director": {
                    "valor": fijos.iloc[2,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                },
                "vigente": 1,
                "categoria": {
                    "valor": data[1].T.reset_index().T.dropna().iloc[1][1].strip()
                },
                "horario_cierre": {
                    "valor": auxhora,
                },
                "campos_faltantes": [],
                "serie":[
                ]
            }
        }


        aux=[]
        for i in range((pdf.getNumPages())//2):
            

            serie=" ".join(data[(7*i)+2].T.reset_index().iloc[1:,1]).strip()
            adqui=" ".join(data[(7*i)+2].T.reset_index().iloc[1:,2]).strip()
            limites=" ".join(data[(7*i)+2].T.reset_index().iloc[1:,3]).strip()
            
            try:
                comifondo=data[(7*i)+4].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
            except:
                comifondo=None
                
            try:
                comicliente=data[(7*i)+3].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
            except:
                comicliente=None
            try:    
                if "hasta" in limites.lower():
                    limsup=limites[limites.lower().find("hasta")+5:].strip()
                    liminf=limites[limites.lower().find("$"):limites.lower().find("hasta")].strip()
                elif "en adelante" in limites.lower():
                    limsup=None
                    liminf=limites[:limites.lower().find("en adelante")].replace("Desde","").replace("desde","").replace("DESDE","").strip()
                else:
                    limsup=None
                    liminf=limites
            except:
                limsup=None
                liminf=limites
                
                
            aux.append({"nombre": serie,
                     "monto_minimo": {
                         "valor": liminf
                    },
                     "monto_maximo": {
                         "valor": limsup
                    },
                     "comision_cliente": {
                         "valor": comicliente
                    },
                      "comision_fondo": {
                         "valor": comifondo
                    },
                     "posibles_adquirientes": {
                         "valor": adqui
                    },
                     "vigente": 1
                    ,
                     "clave_BMV": RES["fondo"]["nombre"]+serie,
                    "campos_faltantes": []
                    })
        RES["fondo"]["serie"]=aux  

        # Guardamos el json final 
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\Jsons\\"+fecha)
        with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"success {p}")
    except:
        try:
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\Templates")

            rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SCOTIA\\"+fecha+"\\DicisSCOTIA\\"+p
            rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_Template2.json"
            rutatemplateaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_Template22.json"
            rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_TemplateFull.json"

            pdf = PyPDF2.PdfFileReader(rutapdf)


            full_temp = list()
            for n_pags in range((pdf.getNumPages()-2)//2):
                with open(rutatemplateaux) as f:
                    temp = json.load(f)
                z = temp.copy()
                [x.update({"page":x["page"]+2*n_pags}) for x in z]
                full_temp.extend(z)

                        #guardamos el nuevo template
            with open("SCOTIA_TemplateFull.json", "w") as f:
                json.dump(full_temp,f)

            data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
            data2=tabula.read_pdf_with_template(rutapdf,rutafull)

            if "variable" in data[1].T.reset_index().T.dropna().iloc[0][1].lower():
                tipoaux=101
            elif "deuda" in data[1].T.reset_index().T.dropna().iloc[0][1].lower():
                tipoaux=100 


            fech=data[2].T.reset_index().T.dropna().iloc[0][1].strip()

            try:
                dia=fech.strip()[:2]
                mes=meses[fech.strip()[2:-4].strip()]
                año=fech.strip()[-4:].strip()
                auxfech=año+mes+dia
            except:
                auxfech=None

            auxhora=" ".join(data[5].T.reset_index().T.dropna().iloc[1:,1])

            if("-" in auxhora or (("hasta" in auxhora.lower()) and ("-" not in auxhora.lower()))):
                auxhora=auxhora[auxhora.lower().find("hasta"):][auxhora[auxhora.lower().find("hasta"):].find(":")-3:].strip()
            elif("hasta" not in auxhora.lower()):
                auxhora=auxhora[auxhora.find(" a ")+2:].strip().replace(" a ","")
                
            razon=" ".join(data[0].T.reset_index().T.iloc[:,0])
            
            permaux=" ".join(data[6].T.reset_index().iloc[1:,0])
        
            if(("no tiene" in permaux.lower()) or ("no se tiene" in permaux.lower()) ):
                permaux="No se tiene plazo mínimo de permanencia"
                
            auxcalif=data[1].T.reset_index().T.dropna().iloc[len(data[1].dropna())][1].strip()  
        
            if(len(auxcalif)<15):
                auxcalif=auxcalif[:auxcalif.find("/")].replace("f","").replace("F","")+auxcalif[auxcalif.find("/"):].replace("s","").replace("S","").replace("(mex)","")
            else:
                auxcalif=auxcalif[:auxcalif.find("/")].replace("f","").replace("F","")+auxcalif[auxcalif.find("/"):auxcalif.find("/")+8].replace("s","").replace("S","").replace("(mex)","")

            RES={
              "fondo": {
                    "nombre": data[1].T.reset_index().T.dropna().iloc[2][1].strip(),
                    "url":  fijos.iloc[0,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                    "razon_social": {
                        "valor": razon[:razon.lower().find("admin")].strip()
                    },
                    "fecha_autorizacion": {
                        "valor": auxfech
                    },
                    "tipo": {
                        "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip(),
                        "id": tipoaux
                    },
                    "administradora": {
                        "nombre":razon[razon.lower().find("admin")+16:].strip(),
                        "valor": 8
                    },
                    "horizonte_inversion": {
                        "valor":  fijos.iloc[1,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                    },
                    "liquidez": {
                        "valor": " ".join(data[5].T.reset_index().iloc[1:,0])
                    },
                    "permanencia": {
                        "valor": permaux
                    },
                    "calificacion_mercado": {
                        "valor": auxcalif
                    },
                    "director": {
                        "valor": fijos.iloc[2,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                    },
                    "vigente": 1,
                    "categoria": {
                        "valor": data[1].T.reset_index().T.dropna().iloc[1][1].strip()
                    },
                    "horario_cierre": {
                        "valor": auxhora,
                    },
                    "campos_faltantes": [],
                    "serie":[
                    ]
                }
            }

            aux=[]
            serie=" ".join(data[(7*0)+2].T.reset_index().iloc[1:,1]).strip()
            adqui=" ".join(data[(7*0)+2].T.reset_index().iloc[1:,2]).strip()
            limites=" ".join(data[(7*0)+2].T.reset_index().iloc[1:,3]).strip()

            try:
                comifondo=data[(7*0)+4].T.reset_index().T.iloc[0:,0][0].strip().replace("$","")
            except:
                comifondo=None

            try:
                comicliente=data[(7*0)+3].T.reset_index().T.iloc[0:,0][0].strip().replace("$","")
            except:
                comicliente=None
            try:    
                if "hasta" in limites.lower():
                    limsup=limites[limites.lower().find("hasta")+5:].strip()
                    liminf=limites[limites.lower().find("$"):limites.lower().find("hasta")].strip()
                elif "en adelante" in limites.lower():
                    limsup=None
                    liminf=limites[:limites.lower().find("en adelante")].replace("Desde","").replace("desde","").replace("DESDE","").strip()
                else:
                    limsup=None
                    liminf=limites
            except:
                limsup=None
                liminf=limites


            aux.append({"nombre": serie,
                         "monto_minimo": {
                             "valor": liminf
                        },
                         "monto_maximo": {
                             "valor": limsup
                        },
                         "comision_cliente": {
                             "valor": comicliente
                        },
                          "comision_fondo": {
                             "valor": comifondo
                        },
                         "posibles_adquirientes": {
                             "valor": adqui
                        },
                         "vigente": 1
                        ,
                         "clave_BMV": RES["fondo"]["nombre"]+serie,
                        "campos_faltantes": []
                        })

            for i in range((pdf.getNumPages()-2)//2):


                serie=" ".join(data2[(3*i)].T.reset_index().iloc[1:,1]).strip()
                adqui=" ".join(data2[(3*i)].T.reset_index().iloc[1:,2]).strip()
                limites=" ".join(data2[(3*i)].T.reset_index().iloc[1:,3]).strip()

                try:
                    comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
                except:
                    comifondo=None

                try:
                    comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
                except:
                    comicliente=None
                try:    
                    if "hasta" in limites.lower():
                        limsup=limites[limites.lower().find("hasta")+5:].strip()
                        liminf=limites[limites.lower().find("$"):limites.lower().find("hasta")].strip()
                    elif "en adelante" in limites.lower():
                        limsup=None
                        liminf=limites[:limites.lower().find("en adelante")].replace("Desde","").replace("desde","").replace("DESDE","").strip()
                    else:
                        limsup=None
                        liminf=limites
                except:
                    limsup=None
                    liminf=limites


                aux.append({"nombre": serie,
                         "monto_minimo": {
                             "valor": liminf
                        },
                         "monto_maximo": {
                             "valor": limsup
                        },
                         "comision_cliente": {
                             "valor": comicliente
                        },
                          "comision_fondo": {
                             "valor": comifondo
                        },
                         "posibles adquirientes": {
                             "valor": adqui
                        },
                         "vigente": 1
                        ,
                         "clave_BMV": RES["fondo"]["nombre"]+serie,
                        "campos_faltantes": []
                        })
            RES["fondo"]["serie"]=aux  

            # Guardamos el json final 
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\Jsons\\"+fecha)
            with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
                json.dump(RES, f, indent=2,ensure_ascii=False)
            print(f"success {p}")
        except:
            try:
                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\Templates")

                rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SCOTIA\\"+fecha+"\\DicisSCOTIA\\"+p
                rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_Template3.json"
                rutatemplateaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_Template22.json"
                rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\SCOTIA\\templates\SCOTIA_TemplateFull.json"

                pdf = PyPDF2.PdfFileReader(rutapdf)


                full_temp = list()
                for n_pags in range((pdf.getNumPages()-2)//2):
                    with open(rutatemplateaux) as f:
                        temp = json.load(f)
                    z = temp.copy()
                    [x.update({"page":x["page"]+2*n_pags}) for x in z]
                    full_temp.extend(z)

                            #guardamos el nuevo template
                with open("SCOTIA_TemplateFull.json", "w") as f:
                    json.dump(full_temp,f)

                data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
                data2=tabula.read_pdf_with_template(rutapdf,rutafull)

                if "variable" in data[1].T.reset_index().T.dropna().iloc[0][1].lower():
                    tipoaux=101
                elif "deuda" in data[1].T.reset_index().T.dropna().iloc[0][1].lower():
                    tipoaux=100 


                fech=data[2].T.reset_index().T.dropna().iloc[0][1].strip()

                try:
                    dia=fech.strip()[:2]
                    mes=meses[fech.strip()[2:-4].strip()]
                    año=fech.strip()[-4:].strip()
                    auxfech=año+mes+dia
                except:
                    auxfech=None

                auxhora=" ".join(data[5].T.reset_index().T.dropna().iloc[1:,1])

                if("-" in auxhora or (("hasta" in auxhora.lower()) and ("-" not in auxhora.lower()))):
                    auxhora=auxhora[auxhora.lower().find("hasta"):][auxhora[auxhora.lower().find("hasta"):].find(":")-3:].strip()
                elif("hasta" not in auxhora.lower()):
                    auxhora=auxhora[auxhora.find(" a ")+2:].strip().replace(" a ","")
                    
                razon=" ".join(data[0].T.reset_index().T.iloc[:,0])
                permaux=" ".join(data[6].T.reset_index().iloc[1:,0])
        
                if(("no tiene" in permaux.lower()) or ("no se tiene" in permaux.lower()) ):
                    permaux="No se tiene plazo mínimo de permanencia"
                    
                auxcalif=data[1].T.reset_index().T.dropna().iloc[len(data[1].dropna())][1].strip()
        
                if(len(auxcalif)<15):
                    auxcalif=auxcalif[:auxcalif.find("/")].replace("f","").replace("F","")+auxcalif[auxcalif.find("/"):].replace("s","").replace("S","").replace("(mex)","")
                else:
                    auxcalif=auxcalif[:auxcalif.find("/")].replace("f","").replace("F","")+auxcalif[auxcalif.find("/"):auxcalif.find("/")+8].replace("s","").replace("S","").replace("(mex)","")

                RES={
                  "fondo": {
                        "nombre": data[1].T.reset_index().T.dropna().iloc[2][1].strip(),
                        "url": fijos.iloc[0,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                        "razon_social": {
                            "valor": razon[:razon.lower().find("admin")].strip()
                        },
                        "fecha_autorizacion": {
                            "valor": auxfech
                        },
                        "tipo": {
                            "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip(),
                            "id": tipoaux
                        },
                        "administradora": {
                            "nombre":razon[razon.lower().find("admin")+16:].strip(),
                            "valor": 8
                        },
                        "horizonte_inversion": {
                            "valor":  fijos.iloc[1,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                        },
                        "liquidez": {
                            "valor": " ".join(data[5].T.reset_index().iloc[1:,0])
                        },
                        "permanencia": {
                            "valor": permaux
                        },
                        "calificacion_mercado": {
                            "valor":auxcalif
                        },
                        "director": {
                            "valor": fijos.iloc[2,list(fijos.columns).index(data[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                        },
                        "vigente": 1,
                        "categoria": {
                            "valor": data[1].T.reset_index().T.dropna().iloc[1][1].strip()
                        },
                        "horario_cierre": {
                            "valor": auxhora,
                        },
                        "campos_faltantes": [],
                        "serie":[
                        ]
                    }
                }

                aux=[]
                serie=" ".join(data[(7*0)+2].T.reset_index().iloc[1:,1]).strip()
                adqui=" ".join(data[(7*0)+2].T.reset_index().iloc[1:,2]).strip()
                limites=" ".join(data[(7*0)+2].T.reset_index().iloc[1:,3]).strip()

                try:
                    comifondo=data[(7*0)+4].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
                except:
                    comifondo=None

                try:
                    comicliente=data[(7*0)+3].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
                except:
                    comicliente=None
                try:    
                    if "hasta" in limites.lower():
                        limsup=limites[limites.lower().find("hasta")+5:].strip()
                        liminf=limites[limites.lower().find("$"):limites.lower().find("hasta")].strip()
                    elif "en adelante" in limites.lower():
                        limsup=None
                        liminf=limites[:limites.lower().find("en adelante")].replace("Desde","").replace("desde","").strip()
                    else:
                        limsup=None
                        liminf=limites
                except:
                    limsup=None
                    liminf=limites


                aux.append({"nombre": serie,
                             "monto_minimo": {
                                 "valor": liminf
                            },
                             "monto_maximo": {
                                 "valor": limsup
                            },
                             "comision_cliente": {
                                 "valor": comicliente
                            },
                              "comision_fondo": {
                                 "valor": comifondo
                            },
                             "posibles_adquirientes": {
                                 "valor": adqui
                            },
                             "vigente": 1
                            ,
                             "clave_BMV": RES["fondo"]["nombre"]+serie,
                            "campos_faltantes": []
                            })

                for i in range((pdf.getNumPages()-2)//2):


                    serie=" ".join(data2[(3*i)].T.reset_index().iloc[1:,1]).strip()
                    adqui=" ".join(data2[(3*i)].T.reset_index().iloc[1:,2]).strip()
                    limites=" ".join(data2[(3*i)].T.reset_index().iloc[1:,3]).strip()

                    try:
                        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
                    except:
                        comifondo=None

                    try:
                        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip().replace("$","")
                    except:
                        comicliente=None
                    try:    
                        if "hasta" in limites.lower():
                            limsup=limites[limites.lower().find("hasta")+5:].strip()
                            liminf=limites[limites.lower().find("$"):limites.lower().find("hasta")].strip()
                        elif "en adelante" in limites.lower():
                            limsup=None
                            liminf=limites[:limites.lower().find("en adelante")].replace("Desde","").replace("desde","").replace("DESDE","").strip()
                        else:
                            limsup=None
                            liminf=limites
                    except:
                        limsup=None
                        liminf=limites


                    aux.append({"nombre": serie,
                             "monto_minimo": {
                                 "valor": liminf
                            },
                             "monto_maximo": {
                                 "valor": limsup
                            },
                             "comision_cliente": {
                                 "valor": comicliente
                            },
                              "comision_fondo": {
                                 "valor": comifondo
                            },
                             "posibles_adquirientes": {
                                 "valor": adqui
                            },
                             "vigente": 1
                            ,
                             "clave_BMV": RES["fondo"]["nombre"]+serie,
                            "campos_faltantes": []
                            })
                RES["fondo"]["serie"]=aux  

                # Guardamos el json final 
                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SCOTIA\Jsons\\"+fecha)
                with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
                    json.dump(RES, f, indent=2,ensure_ascii=False)
                print(f"success {p}")
            except Exception as e:
                os.chdir(direcactual)
                print(f"error enfondo {p}")
                print(e)
                error.append(p)


success FINDE1.pdf
success FIRMA-D.pdf
success FIRMA-E.pdf
success MG-INTL.pdf
success SBANK50.pdf
success SBANKCP.pdf
success SBANKDL.pdf
success SBANKMP.pdf
success SCOT-03.pdf
success SCOT-04.pdf
success SCOT-22.pdf
success SCOT-29.pdf
success SCOT-36.pdf
success SCOT-43.pdf
success SCOT-50.pdf
success SCOT-FR.pdf
success SCOT-FX.pdf
success SCOT-GLO.pdf
success SCOT-RV.pdf
success SCOT-TR.pdf
success SCOT100.pdf
success SCOT200.pdf
success SCOTDL+.pdf
success SCOTDOL.pdf
success SCOTEUR.pdf
success SCOTGL+.pdf
success SCOTI10.pdf
success SCOTI12.pdf
success SCOTI14.pdf
success SCOTIA1.pdf
success SCOTIA2.pdf
success SCOTIAG.pdf
success SCOTILP.pdf
success SCOTIMB.pdf
success SCOTLPG.pdf
success SCOTQNT.pdf
success SCOTUDI.pdf
success SCOTUSA.pdf


# Templeton

In [ ]:
value

# Vector


In [ ]:
#VECTPA

import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]


try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons")
    #os.mkdir(str(datetime.date.today()))
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))



    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\\VECTORFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VECTOR\\"+str(datetime.date.today())+"\\DicisVECOR""\\"+"VECTPA.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"VECTPA_Template1.json")
    data2= tabula.read_pdf_with_template(rutapdf,ruta+"VECTPA_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)

    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data1[1].T.reset_index().T.iloc[:,1].dropna())[5]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data1[1].T.reset_index().T.iloc[:,1].dropna())[4]
    if len(auxcal)>2:
        if(auxcal.find("/")>0):
            auxcal=auxcal
        else:
            auxcal=auxcal[auxcal.find("mx")+2:][:auxcal[auxcal.find("mx")+2:].find(" ")]+"/"+auxcal[auxcal.find("mx")+2:][auxcal[auxcal.find("mx")+2:].find(" ")+2:auxcal[auxcal.find("mx")+2:].find(" ")+3]
    else:
        auxcal=auxcal

    cierreaux=list(data1[3].T.reset_index().T.iloc[:,1].dropna())[1]
    cierreaux="Compra: "+cierreaux[cierreaux.lower().find("compra"):][cierreaux[cierreaux.lower().find("compra"):].find("-")+1:cierreaux[cierreaux.lower().find("compra"):].find("-")+6].strip()+" - Venta: "+cierreaux[cierreaux.lower().find("venta"):][cierreaux[cierreaux.lower().find("venta"):].find("-")+1:cierreaux[cierreaux.lower().find("venta"):].find("-")+6].strip()    

    RES={
                              "fondo": {
                                    "nombre": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.find("INVER")+9:razon.find("Admin")].strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("Vect"):].strip(),
                                        "valor": 80
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data1[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal
                                    },
                                    "director": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[2:,1].dropna())
            serie=list(data2[3*i].T.reset_index().T.iloc[3:,0].dropna())[0]
            limites=" ".join(list(data2[3*i].T.reset_index().T.iloc[3:,2].dropna()))
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()
        except:
            adqui=data2[len(data2)-3].T.reset_index().T.iloc[2,1]
            serie=data2[len(data2)-3].T.reset_index().T.iloc[2,0]
            limites=data2[len(data2)-3].T.reset_index().T.iloc[2,2]
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"VECTPA.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success VECTPA")
except Exception as e:
    print(f"error enfondo VECTPA")
    print(e)
    error.append("VECTPA")
    
#VECTPRE

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons")
    #os.mkdir(str(datetime.date.today()))
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))




    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\\VECTORFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VECTOR\\"+str(datetime.date.today())+"\\DicisVECOR""\\"+"VECTPRE.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"VECTPRE_Template1.json")
    data2= tabula.read_pdf_with_template(rutapdf,ruta+"VECTPRE_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)

    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()


    try:
        if "variable" in data1[1].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data1[1].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=data1[1].T.reset_index().T.dropna().iloc[len(data1[1])][1].strip().replace("\r"," ")

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=data1[1].T.reset_index().T.dropna().iloc[len(data1[1])-1][1].strip()
    if len(auxcal)>2:
        if(auxcal.find("/")>0):
            auxcal=auxcal
        else:
            auxcal=auxcal[auxcal.find("mx")+2:][:auxcal[auxcal.find("mx")+2:].find(" ")]+"/"+auxcal[auxcal.find("mx")+2:][auxcal[auxcal.find("mx")+2:].find(" ")+2:auxcal[auxcal.find("mx")+2:].find(" ")+3]
    else:
        auxcal=auxcal

    cierreaux=" ".join(data1[3].T.reset_index().T.iloc[1:,1].dropna())
    cierreaux="Compra: "+cierreaux[cierreaux.lower().find("compra"):][cierreaux[cierreaux.lower().find("compra"):].find("-")+1:cierreaux[cierreaux.lower().find("compra"):].find("-")+7].strip()+" - Venta: "+cierreaux[cierreaux.lower().find("venta"):][cierreaux[cierreaux.lower().find("venta"):].find("-")+1:cierreaux[cierreaux.lower().find("venta"):].find("-")+7].strip()  



    RES={
                              "fondo": {
                                    "nombre": data1[1].T.reset_index().T.dropna().iloc[2][1].strip(),
                                    "url":fijos.iloc[0,list(fijos.columns).index(data1[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.find("INVER")+9:razon.find("Admin")].strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor": auxfech
                                    },
                                    "tipo": {
                                        "valor": data1[1].T.reset_index().T.dropna().iloc[0][1].strip(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("Vect"):].strip(),
                                        "valor": 80
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(data1[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "liquidez": {
                                        "valor":data1[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal
                                    },
                                    "director": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(data1[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[1].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):

        adqui=" ".join(data2[(3*9)].T.reset_index().T.iloc[2:,1].dropna())
        serie=list(data2[3*i].T.reset_index().T.iloc[3:,0].dropna())[0]
        limites=" ".join(list(data2[3*i].T.reset_index().T.iloc[3:,2].dropna()))
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))



    with open(fecha+"_"+"VECTPRE.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)

    print("success VECTPRE")
except Exception as e:
    print(f"error enfondo VECTPRE")
    print(e)
    error.append("VECTPRE")
    
#VECTPZO

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons")
    #os.mkdir(str(datetime.date.today()))
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))



    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\\VECTORFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VECTOR\\"+str(datetime.date.today())+"\\DicisVECOR""\\"+"VECTPZO.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"VECTPZO_Template1.json")
    data2= tabula.read_pdf_with_template(rutapdf,ruta+"VECTPZO_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)

    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()


    try:
        if "variable" in list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data1[1].T.reset_index().T.iloc[:,1].dropna())[len(data1[1])]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3]
    if len(auxcal)>2:
        if(auxcal.find("/")>0):
            auxcal=auxcal
        else:
            auxcal=auxcal[auxcal.find("mx")+2:][:auxcal[auxcal.find("mx")+2:].find(" ")]+"/"+auxcal[auxcal.find("mx")+2:][auxcal[auxcal.find("mx")+2:].find(" ")+2:auxcal[auxcal.find("mx")+2:].find(" ")+3]
    else:
        auxcal=auxcal

    cierreaux=" ".join(data1[3].T.reset_index().T.iloc[2:,1].dropna())
    cierreaux="Compra: "+cierreaux[cierreaux.lower().find("compra"):][cierreaux[cierreaux.lower().find("compra"):].find("las")+4:cierreaux[cierreaux.lower().find("compra"):].find("las")+9].strip()+" - Venta: "+cierreaux[cierreaux.lower().find("venta"):][cierreaux[cierreaux.lower().find("venta"):].find("las")+4:cierreaux[cierreaux.lower().find("venta"):].find("las")+9].strip() 

    RES={
                              "fondo": {
                                    "nombre": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.find("INVER")+9:razon.find("Admin")].strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("Vect"):].strip(),
                                        "valor": 80
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data1[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal
                                    },
                                    "director": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[1]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[2:,1].dropna())
            serie=list(data2[3*i].T.reset_index().T.iloc[3:,0].dropna())[0]
            limites=" ".join(list(data2[3*i].T.reset_index().T.iloc[3:,2].dropna()))
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()
        except:
            adqui=" ".join(data2[len(data2)-3].T.reset_index().T.iloc[2:,1].dropna())
            serie=data2[len(data2)-3].T.reset_index().T.iloc[2,0]
            limites=data2[len(data2)-3].T.reset_index().T.iloc[2,2]
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))



    with open(fecha+"_"+"VECTPZO.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success VECTPZO")
except Exception as e:
    print(f"error enfondo VECTPZO")
    print(e)
    error.append("VECTPZO")    
#VECTRF

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons")
    #os.mkdir(str(datetime.date.today()))
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))




    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\\VECTORFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VECTOR\\"+str(datetime.date.today())+"\\DicisVECOR""\\"+"VECTRF.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"VECTRF_Template1.json")
    data2= tabula.read_pdf_with_template(rutapdf,ruta+"VECTRF_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)

    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()


    try:
        if "variable" in data1[1].T.reset_index().T.dropna().iloc[0][1].strip().strip().lower():
            tipoaux=101
        elif "deuda" in data1[1].T.reset_index().T.dropna().iloc[0][1].strip().strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=data1[1].T.reset_index().T.dropna().iloc[len(data1[1])][1].strip().replace("\r"," ")

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=data1[1].T.reset_index().T.dropna().iloc[len(data1[1])-1][1].strip()
    if len(auxcal)>2:
        if(auxcal.find("/")>0):
            auxcal=auxcal
        else:
            auxcal=auxcal[auxcal.find("mx")+2:][:auxcal[auxcal.find("mx")+2:].find(" ")]+"/"+auxcal[auxcal.find("mx")+2:][auxcal[auxcal.find("mx")+2:].find(" ")+2:auxcal[auxcal.find("mx")+2:].find(" ")+3]
    else:
        auxcal=auxcal

    cierreaux=" ".join(data1[3].T.reset_index().T.iloc[:,1].dropna()[1:])[" ".join(data1[3].T.reset_index().T.iloc[:,1].dropna()[1:]).find(".")+1:].strip()
    cierreaux="Compra: "+cierreaux[cierreaux.lower().find("compra"):][cierreaux[cierreaux.lower().find("compra"):].find("las")+4:cierreaux[cierreaux.lower().find("compra"):].find("las")+10].strip()+" - Venta: "+cierreaux[cierreaux.lower().find("venta"):][cierreaux[cierreaux.lower().find("venta"):].find("las")+4:cierreaux[cierreaux.lower().find("venta"):].find("las")+10].strip() 

    RES={
                              "fondo": {
                                    "nombre": data1[1].T.reset_index().T.dropna().iloc[2][1].strip(),
                                    "url":fijos.iloc[0,list(fijos.columns).index(data1[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.find("INVER")+9:razon.find("Admin")].strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor": auxfech
                                    },
                                    "tipo": {
                                        "valor": data1[1].T.reset_index().T.dropna().iloc[0][1].strip(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("Vect"):].strip(),
                                        "valor": 80
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(data1[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "liquidez": {
                                        "valor":data1[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal
                                    },
                                    "director": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(data1[1].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[1].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[2:,1].dropna())
            serie=list(data2[3*i].T.reset_index().T.iloc[3:,0].dropna())[0]
            limites=" ".join(list(data2[3*i].T.reset_index().T.iloc[3:,2].dropna()))
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()
        except:
            adqui=" ".join(data2[len(data2)-3].T.reset_index().T.iloc[2:,1].dropna())
            serie=data2[len(data2)-3].T.reset_index().T.iloc[2,0]
            limites=data2[len(data2)-3].T.reset_index().T.iloc[2,2]
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))



    with open(fecha+"_"+"VECTRF.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success VECTRF")
except Exception as e:
    print(f"error enfondo VECTRF")
    print(e)
    error.append("VECTRF")     
#VECTSIC


try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons")
    #os.mkdir(str(datetime.date.today()))
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))


    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\\VECTORFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VECTOR\\"+str(datetime.date.today())+"\\DicisVECOR""\\"+"VECTSIC.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"VECTSIC_Template1.json")
    data2= tabula.read_pdf_with_template(rutapdf,ruta+"VECTSIC_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)

    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data1[1].T.reset_index().T.iloc[:,1].dropna())[5]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data1[1].T.reset_index().T.iloc[:,1].dropna())[4]
    if len(auxcal)>2:
        if(auxcal.find("/")>0):
            auxcal=auxcal
        else:
            auxcal=auxcal[auxcal.find("mx")+2:][:auxcal[auxcal.find("mx")+2:].find(" ")]+"/"+auxcal[auxcal.find("mx")+2:][auxcal[auxcal.find("mx")+2:].find(" ")+2:auxcal[auxcal.find("mx")+2:].find(" ")+3]
    else:
        auxcal=auxcal

    cierreaux=data1[3].T.reset_index().T.iloc[:,1].dropna()[2]
    cierreaux="Compras: "+ cierreaux[cierreaux.lower().find("compras")+7:cierreaux.lower().find("compras")+13].strip()+" - "+"Ventas: "+cierreaux[cierreaux.lower().find("ventas")+7:].strip()    


    RES={
                              "fondo": {
                                    "nombre": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.find("INVER")+9:razon.find("Admin")].strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("Vect"):].strip(),
                                        "valor": 80
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data1[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal
                                    },
                                    "director": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data1[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data2[(3 * i)].T.reset_index().T.iloc[2:,1].dropna())
            serie=list(data2[3*i].T.reset_index().T.iloc[3:,0].dropna())[0]
            limites=" ".join(list(data2[3*i].T.reset_index().T.iloc[3:,2].dropna()))
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()
        except:
            adqui=" ".join(data2[len(data2)-3].T.reset_index().T.iloc[2:,1].dropna())
            serie=data2[len(data2)-3].T.reset_index().T.iloc[2,0]
            limites=data2[len(data2)-3].T.reset_index().T.iloc[2,2]
            comicliente=data2[(3*i)+1].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data2[(3*i)+2].T.reset_index().T.iloc[0:,1][0].strip()

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VECTOR\Jsons"+"\\"+str(datetime.date.today()))


    with open(fecha+"_"+"VECTSIC.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success VECTSIC")
except Exception as e:
    print(f"error enfondo VECTSIC")
    print(e)
    error.append("VECTSIC")     

# Valmex

In [60]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

fecha=str(datetime.date.today())
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALMEX\\"+fecha+"\DicisVALMEX")
docs=os.listdir()
error=[]
fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\\VALMEXFijos.csv")
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
    "octubre":"10","noviembre":"11","diciembre":"12"}


for p in docs:
    try:
        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALMEX\\"+fecha+"\DicisVALMEX\\"+ p
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates\\VALMX_TemplateGral.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates\\VALMX_TemplateGralFull.json"

        pdf = PyPDF2.PdfFileReader(rutapdf)

        full_temp = list()
        for n_pags in range((pdf.getNumPages())//2):
            with open(rutatemplate) as f:
                temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

        with open("VALMX_TemplateGralFull.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutafull)


        try:
            if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                tipoaux=101
            elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                tipoaux=100
        except:
            tipoaux=None


        fech=data[0].T.reset_index().T.dropna().iloc[len(data[0])][1].strip()

        try:
            dia=fech.strip()[:2]
            mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
            año=fech.strip()[-4:].strip()
            auxfech=año+mes+dia
        except:
            auxfech=None    
        
        auxcali=data[0].T.reset_index().T.dropna().iloc[len(data[0])-1][1].strip()
        if(len(auxcali)>15):
            auxcali=auxcali[auxcali.lower().find(",")+1:auxcali.lower().find("escala")].strip()
        else:
            auxcali=auxcali

        RES={
                  "fondo": {
                    "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                     "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                      "razon_social": {
                        "valor": data[2].T.reset_index().T.iloc[:,0].dropna().values[0].replace("/","")
                             },
                        "fecha_autorizacion": {
                         "valor": auxfech
                        },
                        "tipo": {
                          "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                              "id": tipoaux
                             },
                         "administradora": {
                           "nombre":data[7].columns[0],
                             "valor": 23
                           },
                            "horizonte_inversion": {
                             "valor": fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                             },
                              "liquidez": {
                               "valor": " ".join(data[5].T.reset_index().T.iloc[:,1].dropna()[:-1])
                               },
                                 "permanencia": {
                                "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                  },
                                 "calificacion_mercado": {
                                  "valor": auxcali
                                    },
                                    "director": {
                                    "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                },
                                "vigente": 1,
                                 "categoria": {
                                     "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                },
                                "horario_cierre": {
                                "valor": data[5].T.reset_index().T.dropna().iloc[1][1][data[5].T.reset_index().T.dropna().iloc[1][1].find("-")+1:].strip(),
                                },
                                 "campos_faltantes": [],
                                "serie":[
                                ]
                            }
                         }

        aux=[]
        for i in range((pdf.getNumPages())//2):
            adqui=" ".join(data[(8*i)+1].T.reset_index().T.iloc[3:,1].dropna())
            serie=list(data[(8*i)+1].T.reset_index().T.iloc[1:,0].dropna())[len(list(data[(8*i)+1].T.reset_index().T.iloc[1:,0].dropna()))-1]
            limites=" ".join(data[(8*i)+1].T.reset_index().T.iloc[3:,2].dropna())
            comicliente=data[(8*i)+3].T.reset_index().T.iloc[0:,1][0].strip()
            comifondo=data[(8*i)+4].T.reset_index().T.iloc[0:,1][0].strip()
            
            try:
                comicliente=comicliente.replace("%","")
            except:
                comicliente=comicliente
            
            try:
                comifondo=comifondo.replace("%","")
            except:
                comifondo=comifondo
            
            
            aux.append({"nombre": serie,
                                        "monto_minimo": {
                                              "valor": limites
                                         },
                                         "monto_maximo": {
                                             "valor": None
                                                },
                                           "comision_cliente": {
                                              "valor": comicliente
                                                },
                                            "comision_fondo": {
                                              "valor": comifondo
                                                },
                                         "posibles_adquirientes": {
                                             "valor": adqui
                                                },
                                           "vigente":1
                                                ,
                                            "clave_BMV": RES["fondo"]["nombre"]+serie,
                                           "campos_faltantes": []
                                                })
        RES["fondo"]["serie"]=aux

        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Jsons\\"+fecha)

        with open(fecha.replace("-","")+"_"+p[p.find("_")+1:][:p[p.find("_")+1:].find("_")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"success {p}")
    except:
        try:
            direcactual=os.getcwd()
            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates")

            rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALMEX\\"+fecha+"\DicisVALMEX\\"+ p
            rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates\\VALMX_TemplateGral2.json"
            rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates\\VALMX_TemplateGralFull.json"

            pdf = PyPDF2.PdfFileReader(rutapdf)

            full_temp = list()
            for n_pags in range((pdf.getNumPages())//2):
                with open(rutatemplate) as f:
                    temp = json.load(f)
                z = temp.copy()
                [x.update({"page":x["page"]+2*n_pags}) for x in z]
                full_temp.extend(z)

            with open("VALMX_TemplateGralFull.json", "w") as f:
                json.dump(full_temp,f)

            data = tabula.read_pdf_with_template(rutapdf,rutafull)


            try:
                if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                    tipoaux=101
                elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                    tipoaux=100
            except:
                tipoaux=None


            fech=data[0].T.reset_index().T.dropna().iloc[len(data[0])][1].strip()

            try:
                dia=fech.strip()[:2]
                mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
                año=fech.strip()[-4:].strip()
                auxfech=año+mes+dia
            except:
                auxfech=None    

            auxcali=data[0].T.reset_index().T.dropna().iloc[len(data[0])-1][1].strip()
            if(len(auxcali)>15):
                auxcali=auxcali[auxcali.lower().find(",")+1:auxcali.lower().find("escala")].strip()
            else:
                auxcali=auxcali


            RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": data[2].T.reset_index().T.iloc[:,0].dropna().values[0].replace("/","")
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":data[7].columns[0],
                                 "valor": 23
                               },
                                "horizonte_inversion": {
                                 "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": " ".join(data[5].T.reset_index().T.iloc[:,1].dropna()[:-1])
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": auxcali
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[5].T.reset_index().T.dropna().iloc[1][1][data[5].T.reset_index().T.dropna().iloc[1][1].find("-")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

            aux=[]
            for i in range((pdf.getNumPages())//2):
                adqui=" ".join(data[(8*i)+1].T.reset_index().T.iloc[3:,1].dropna())
                serie=list(data[(8*i)+1].T.reset_index().T.iloc[1:,0].dropna())[len(list(data[(8*i)+1].T.reset_index().T.iloc[1:,0].dropna()))-1]
                limites=" ".join(data[(8*i)+1].T.reset_index().T.iloc[3:,2].dropna())
                comicliente=data[(8*i)+3].T.reset_index().T.iloc[0:,1][0].strip()
                comifondo=data[(8*i)+4].T.reset_index().T.iloc[0:,1][0].strip()
                
                try:
                    comicliente=comicliente.replace("%","")
                except:
                    comicliente=comicliente

                try:
                    comifondo=comifondo.replace("%","")
                except:
                    comifondo=comifondo
            

                aux.append({"nombre": serie,
                                            "monto_minimo": {
                                                  "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })
            RES["fondo"]["serie"]=aux

            os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Jsons\\"+fecha)

            with open(fecha.replace("-","")+"_"+p[p.find("_")+1:][:p[p.find("_")+1:].find("_")]+".json", "w", encoding="ansi") as f:
                json.dump(RES, f, indent=2,ensure_ascii=False)
            print(f"success {p}")
        except:
            try:
                direcactual=os.getcwd()
                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates")

                rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALMEX\\"+fecha+"\DicisVALMEX\\"+ p
                rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates\\VALMX_TemplateGral3.json"
                rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Templates\\VALMX_TemplateGralFull.json"

                pdf = PyPDF2.PdfFileReader(rutapdf)

                full_temp = list()
                for n_pags in range((pdf.getNumPages())//2):
                    with open(rutatemplate) as f:
                        temp = json.load(f)
                    z = temp.copy()
                    [x.update({"page":x["page"]+2*n_pags}) for x in z]
                    full_temp.extend(z)

                with open("VALMX_TemplateGralFull.json", "w") as f:
                    json.dump(full_temp,f)

                data = tabula.read_pdf_with_template(rutapdf,rutafull)


                try:
                    if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                        tipoaux=101
                    elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                        tipoaux=100
                except:
                    tipoaux=None


                fech=data[0].T.reset_index().T.dropna().iloc[len(data[0])][1].strip()

                try:
                    dia=fech.strip()[:2]
                    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
                    año=fech.strip()[-4:].strip()
                    auxfech=año+mes+dia
                except:
                    auxfech=None   
                    
                auxcali=data[0].T.reset_index().T.dropna().iloc[len(data[0])-1][1].strip()
                if(len(auxcali)>15):
                    auxcali=auxcali[auxcali.lower().find(",")+1:auxcali.lower().find("escala")].strip()
                else:
                    auxcali=auxcali

                RES={
                          "fondo": {
                            "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                             "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                              "razon_social": {
                                "valor": data[2].T.reset_index().T.iloc[:,0].dropna().values[0].replace("/","")
                                     },
                                "fecha_autorizacion": {
                                 "valor": auxfech
                                },
                                "tipo": {
                                  "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                      "id": tipoaux
                                     },
                                 "administradora": {
                                   "nombre":data[7].columns[0],
                                     "valor": 23
                                   },
                                    "horizonte_inversion": {
                                     "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                     },
                                      "liquidez": {
                                       "valor": " ".join(data[5].T.reset_index().T.iloc[:,1].dropna()[:-1])
                                       },
                                         "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                          },
                                         "calificacion_mercado": {
                                          "valor": auxcali
                                            },
                                            "director": {
                                            "valor":  fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                        },
                                        "vigente": 1,
                                         "categoria": {
                                             "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                        },
                                        "horario_cierre": {
                                        "valor": data[5].T.reset_index().T.dropna().iloc[1][1][data[5].T.reset_index().T.dropna().iloc[1][1].find("-")+1:].strip(),
                                        },
                                         "campos_faltantes": [],
                                        "serie":[
                                        ]
                                    }
                                 }

                aux=[]
                for i in range((pdf.getNumPages())//2):
                    adqui=" ".join(data[(8*i)+1].T.reset_index().T.iloc[3:,1].dropna())
                    serie=list(data[(8*i)+1].T.reset_index().T.iloc[1:,0].dropna())[len(list(data[(8*i)+1].T.reset_index().T.iloc[1:,0].dropna()))-1]
                    limites=" ".join(data[(8*i)+1].T.reset_index().T.iloc[3:,2].dropna())
                    comicliente=data[(8*i)+3].T.reset_index().T.iloc[0:,1][0].strip()
                    comifondo=data[(8*i)+4].T.reset_index().T.iloc[0:,1][0].strip()

                    try:
                        comicliente=comicliente.replace("%","")
                    except:
                        comicliente=comicliente

                    try:
                        comifondo=comifondo.replace("%","")
                    except:
                        comifondo=comifondo
            
                    aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
                RES["fondo"]["serie"]=aux

                os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALMEX\Jsons\\"+fecha)

                with open(fecha.replace("-","")+"_"+p[p.find("_")+1:][:p[p.find("_")+1:].find("_")]+".json", "w", encoding="ansi") as f:
                    json.dump(RES, f, indent=2,ensure_ascii=False)
                print(f"success {p}")
            except Exception as e:
                os.chdir(direcactual)
                print(f"error enfondo {p}")
                print(e)
                error.append(p)

success DICI_BX+0_0721.pdf
success DICI_BX+1_0721.pdf
success DICI_BX+2_0721.pdf
success DICI_BX+3_0721.pdf
success DICI_VALMX10_0621.pdf
success DICI_VALMX14_0621.pdf
success DICI_VALMX15_0621.pdf
success DICI_VALMX16_0621.pdf
success DICI_VALMX17_0621.pdf
success DICI_VALMX19_0621.pdf
success DICI_VALMX20_0621.pdf
success DICI_VALMX28_0621.pdf
success DICI_VALMX30_0621.pdf
success DICI_VALMX34_0621.pdf
success DICI_VALMXA_0621.pdf
success DICI_VALMXES_0621.pdf
success DICI_VALMXRP_0621.pdf
success DICI_VALMXVL_0621.pdf
success DICI_VLMXESG_0621.pdf
success DICI_VLMXETF_0621.pdf
success DICI_VLMXFLX_0621.pdf
success DICI_VLMXJUB_0621.pdf
success DICI_VLMXLPE_0621.pdf
success DICI_VLMXP24_0621.pdf
success DICI_VLMXP31_0621.pdf
success DICI_VLMXP38_0621.pdf
success DICI_VLMXP45_0621.pdf
success DICI_VLMXP52_0621.pdf
success DICI_VLMXTEC_0621.pdf


# Skandia


In [ ]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime
import warnings
warnings.filterwarnings("ignore")

meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}
error=[]

try:
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\\SKANDIAFijos.csv")

    fecha=str(datetime.date.today())
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SKANDIA\\"+fecha+"\\DicisSKANDIA\\"+"SK-DCP.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"SK-DCP_Template1.json")
    data2=tabula.read_pdf_with_template(rutapdf,ruta+"SK-DCP_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)


    fech=data1[0].T.reset_index().T.iloc[:,1].dropna()[3]
    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None


    try:
        if "variable" in list(data1[0].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data1[0].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    razon=' '.join( str(x) for x in list(data1[1].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()
    
    caliaux=data1[0].T.reset_index().T.iloc[:,1].dropna()[2]
    if("HR" in caliaux):
        caliaux=(caliaux[caliaux.find("/")-4:caliaux.find("/")]+caliaux[caliaux.find("/"):caliaux.find("/")+2]).strip()
    else:
        caliaux=caliaux
    
    RES={
                              "fondo": {
                                    "nombre": data1[0].T.reset_index().T.iloc[:,1].dropna()[1],
                                    "url":fijos.iloc[0,list(fijos.columns).index(data1[0].T.reset_index().T.iloc[:,1].dropna()[1])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.find("C.V.")+4]
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[0].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("por")+3:].strip(),
                                        "valor": 65
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(data1[0].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data1[2].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": caliaux
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(data1[0].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[0].T.reset_index().T.iloc[:,1].dropna()[0]
                                    },
                                    "horario_cierre": {
                                        "valor": data1[2].T.reset_index().T.iloc[:,1].dropna()[0][data1[2].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }


    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
        serie=list(data2[5*i].T.reset_index().T.iloc[2:,0].dropna())[0]
        limites=" ".join(list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna()))
        if (i==0):
            comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
        else:
            comicliente=data2[(5*i)+4].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+3].T.reset_index().T[1])[0]
            
        try:
            comicliente=comicliente.replace("%","")
        except:
            comicliente=comicliente
            
        try:
            comifondo=comifondo.replace("%","")
        except:
            comifondo=comifondo

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })
    RES["fondo"]["serie"]=aux
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Jsons\\"+fecha)

    with open(fecha+"_"+"SK-DCP.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success SK-DCP")
except Exception as e:
    error.append("SK-DCP")
    print(f"error enfondo SK-DCP")
    print(e)
    
try:

    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\\SKANDIAFijos.csv")

    fecha=str(datetime.date.today())
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SKANDIA\\"+fecha+"\\DicisSKANDIA\\"+"SK-DEST.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"SK-DEST_Tempate1.json")
    data2=tabula.read_pdf_with_template(rutapdf,ruta+"SK-DEST_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)


    fech=data1[1].T.reset_index().T.iloc[:,1].dropna()[3]
    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None


    try:
        if "variable" in  list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in  list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()
    
    caliaux= data1[1].T.reset_index().T.iloc[:,1].dropna()[2]
    if("HR" in caliaux):
        caliaux=(caliaux[caliaux.find("/")-4:caliaux.find("/")]+caliaux[caliaux.find("/"):caliaux.find("/")+2]).strip()
    else:
        caliaux=caliaux
    RES={
                              "fondo": {
                                    "nombre": data1[1].T.reset_index().T.iloc[:,1].dropna()[1],
                                    "url": fijos.iloc[0,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.find("C.V.")+4]
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("por")+3:].strip(),
                                        "valor": 65
                                    },
                                    "horizonte_inversion": {
                                        "valor":  fijos.iloc[1,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "liquidez": {
                                        "valor":list(data1[2].T.reset_index().T.dropna().iloc[:,0])[0].replace("Liquidez","").strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": caliaux
                                    },
                                    "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[1].T.reset_index().T.iloc[:,1].dropna()[0]
                                    },
                                    "horario_cierre": {
                                        "valor": data1[2].T.reset_index().T.iloc[:,0].dropna()[0][data1[2].T.reset_index().T.iloc[:,0].dropna()[0].find(" a ")+2:].strip(),
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }


    aux=[]
    for i in range((pdf.getNumPages())//2):
        if(i==0):
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)].T.reset_index().T.iloc[2:,0].dropna()))
            
            comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
        else:
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[5*i].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna()))
            comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]

        try:
            comicliente=comicliente.replace("%","")
        except:
            comicliente=comicliente
            
        try:
            comifondo=comifondo.replace("%","")
        except:
            comifondo=comifondo
            
        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })
    RES["fondo"]["serie"]=aux
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Jsons\\"+fecha)

    with open(fecha+"_"+"SK-DEST.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success SK-DEST")
except Exception as e:
    error.append("SK-DEST")
    print(f"error enfondo SK-DEST")
    print(e)
    
try:

    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\\SKANDIAFijos.csv")

    fecha=str(datetime.date.today())
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SKANDIA\\"+fecha+"\\DicisSKANDIA\\"+"SK-GUB.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"SK-GUB_Template1.json")
    data2=tabula.read_pdf_with_template(rutapdf,ruta+"SK-GUB_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)


    fech=data1[1].T.reset_index().T.iloc[:,1].dropna()[3]
    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None


    try:
        if "variable" in  list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in  list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()
    
    caliaux=data1[1].T.reset_index().T.iloc[:,1].dropna()[2]
    if("HR" in caliaux):
        caliaux=(caliaux[caliaux.find("/")-4:caliaux.find("/")]+caliaux[caliaux.find("/"):caliaux.find("/")+2]).strip()
    else:
        caliaux=caliaux
    
    RES={
                              "fondo": {
                                    "nombre": data1[1].T.reset_index().T.iloc[:,1].dropna()[1],
                                    "url":fijos.iloc[0,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.find("C.V.")+4]
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("por")+3:].strip(),
                                        "valor": 65
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "liquidez": {
                                        "valor":list(data1[2].T.reset_index().T.dropna().iloc[:,1])[0].replace("Liquidez","").strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": caliaux
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[1].T.reset_index().T.iloc[:,1].dropna()[0]
                                    },
                                    "horario_cierre": {
                                        "valor": data1[2].T.reset_index().T.iloc[:,1].dropna()[0][data1[2].T.reset_index().T.iloc[:,1].dropna()[0].find(" a ")+2:].strip().strip(),
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }


    aux=[]
    for i in range((pdf.getNumPages())//2):
        if(i==0):
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)].T.reset_index().T.iloc[2:,0].dropna()))
            
            comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
        else:
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[5*i].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna()))
            comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
            
        try:
            comicliente=comicliente.replace("%","")
        except:
            comicliente=comicliente
            
        try:
            comifondo=comifondo.replace("%","")
        except:
            comifondo=comifondo

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })
    RES["fondo"]["serie"]=aux
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Jsons\\"+fecha)

    with open(fecha+"_"+"SK-GUB.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success SK-GUB")
except Exception as e:
    error.append("SK-GUB")
    print(f"error enfondo SK-GUB")
    print(e)
    
try:
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\\SKANDIAFijos.csv")

    fecha=str(datetime.date.today())
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SKANDIA\\"+fecha+"\\DicisSKANDIA\\"+"SK-RVMX.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"SK-RVMX_Template1.json")
    data2=tabula.read_pdf_with_template(rutapdf,ruta+"SK-RVMX_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)


    fech=data1[1].T.reset_index().T.iloc[:,1].dropna()[3]
    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None


    try:
        if "variable" in  list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in  list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()
    
    caliaux=data1[1].T.reset_index().T.iloc[:,1].dropna()[2]
    if("HR" in caliaux):
        caliaux=(caliaux[caliaux.find("/")-4:caliaux.find("/")]+caliaux[caliaux.find("/"):caliaux.find("/")+2]).strip()
    else:
        caliaux=caliaux
        
    RES={
                              "fondo": {
                                    "nombre": data1[1].T.reset_index().T.iloc[:,1].dropna()[1],
                                    "url":fijos.iloc[0,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.find("C.V.")+4]
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data1[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("por")+6:].strip(),
                                        "valor": 65
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "liquidez": {
                                        "valor":list(data1[2].T.reset_index().T.dropna().iloc[:,0])[0].replace("Liquidez","").strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": caliaux
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,1].dropna()[1])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[1].T.reset_index().T.iloc[:,1].dropna()[0]
                                    },
                                    "horario_cierre": {
                                        "valor": data1[2].T.reset_index().T.iloc[:,0].dropna()[0][data1[2].T.reset_index().T.iloc[:,0].dropna()[0].find(" a ")+2:].strip().strip(),
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }


    aux=[]
    for i in range((pdf.getNumPages())//2):
        if(i==0):
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)].T.reset_index().T.iloc[2:,0].dropna()))
            
            comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
        else:
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[5*i].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna()))
            comicliente=data2[(5*i)+4].T.reset_index().T.iloc[:,1][0]
            comifondo=list(data2[(5*i)+3].T.reset_index().T[1])[0]

            
        try:
            comicliente=comicliente.replace("%","")
        except:
            comicliente=comicliente
            
        try:
            comifondo=comifondo.replace("%","")
        except:
            comifondo=comifondo
            
        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })
    RES["fondo"]["serie"]=aux
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Jsons\\"+fecha)

    with open(fecha+"_"+"SK-RVMX.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success SK-RVMX")
except Exception as e:
    error.append("SK-RVMX")
    print(f"error enfondo SK-RVMX")
    print(e)

try:
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\\SKANDIAFijos.csv")

    fecha=str(datetime.date.today())
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Templates\\"
    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\SKANDIA\\"+fecha+"\\DicisSKANDIA\\"+"SK-RVST.pdf"
    data1 = tabula.read_pdf_with_template(rutapdf,ruta+"SK-RVST_Template1.json")
    data2=tabula.read_pdf_with_template(rutapdf,ruta+"SK-RVST_Template2.json")
    pdf = PyPDF2.PdfFileReader(rutapdf)


    fech=data1[1].T.reset_index().T.iloc[:,0].dropna()[4].replace("Fecha de Autorización","").strip()
    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None


    try:
        if "variable" in  " ".join(data1[1].T.reset_index().T.iloc[:,0].values[:1]).replace("Tipo","").replace("Categoría","").strip():
            tipoaux=101
        elif "deuda" in  " ".join(data1[1].T.reset_index().T.iloc[:,0].values[:1]).replace("Tipo","").replace("Categoría","").strip():
            tipoaux=100
    except:
        tipoaux=None


    razon=' '.join( str(x) for x in list(data1[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()
    
    caliaux=data1[1].T.reset_index().T.iloc[:,0].values[4].replace("Calificación","").strip()
    if("HR" in caliaux):
        caliaux=(caliaux[caliaux.find("/")-4:caliaux.find("/")]+caliaux[caliaux.find("/"):caliaux.find("/")+2]).strip()
    else:
        caliaux=caliaux
    
    RES={
                              "fondo": {
                                    "nombre": data1[1].T.reset_index().T.iloc[:,0].values[3].replace("Clave de pizarra","").strip(),
                                    "url":fijos.iloc[0,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,0].values[3].replace("Clave de pizarra","").strip())].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.find("C.V.")+4]
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data1[1].T.reset_index().T.iloc[:,0].values[:1]).replace("Tipo","").replace("Categoría","").strip(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.find("po")+6:].strip(),
                                        "valor": 65
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,0].values[3].replace("Clave de pizarra","").strip())].strip()
                                    },
                                    "liquidez": {
                                        "valor":list(data1[2].T.reset_index().T.dropna().iloc[:,0])[0].replace("Liquidez","").strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data1[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": caliaux
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(data1[1].T.reset_index().T.iloc[:,0].values[3].replace("Clave de pizarra","").strip())].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": data1[1].T.reset_index().T.iloc[:,0].values[2].strip()
                                    },
                                    "horario_cierre": {
                                        "valor": data1[2].T.reset_index().T.iloc[:,0].dropna()[0][data1[2].T.reset_index().T.iloc[:,0].dropna()[0].find(" a ")+2:].strip().strip(),
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }


    aux=[]
    for i in range((pdf.getNumPages())//2):
        if(i==0):
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[(5*i)+2].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)].T.reset_index().T.iloc[3:,0].dropna()))
            try:
                comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0].replace(" ","")
                comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0].replace(" ","")
            except:
                comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
                comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
        else:
            adqui=" ".join(list(data2[(5*i)+1].T.reset_index().T.iloc[1:,0].dropna()))
            serie=list(data2[5*i].T.reset_index().T.iloc[2:,0].dropna())[0]
            limites=" ".join(list(data2[(5*i)+2].T.reset_index().T.iloc[3:,0].dropna()))
            try:
                comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0].replace(" ","")
                comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0].replace(" ","")
            except:
                comicliente=data2[(5*i)+3].T.reset_index().T.iloc[:,1][0]
                comifondo=list(data2[(5*i)+4].T.reset_index().T[1])[0]
                
                
        try:
            comicliente=comicliente.replace("%","")
        except:
            comicliente=comicliente
            
        try:
            comifondo=comifondo.replace("%","")
        except:
            comifondo=comifondo
            
            
        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": limites
                                             },
                                             "monto_maximo": {
                                                 "valor": None
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente": 1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })
    RES["fondo"]["serie"]=aux
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\SKANDIA\Jsons\\"+fecha)

    with open(fecha+"_"+"SK-RVST.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success SK-RVST")
except Exception as e:
    error.append("SK-RVST")
    print(f"error enfondo SK-RVST")
    print(e)

# FINAMEX


In [ ]:
#Checar el plazo minimo de permanencia

In [ ]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime



fecha=str(datetime.date.today())
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\FINAMEX\\"+fecha+"\DicisFINAMEX")
docs=["FINADIN.pdf"]#os.listdir()
error=[]
fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\FINAMEX\\FINAMEXFijos.csv")
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
    "octubre":"10","noviembre":"11","diciembre":"12"}

error=[]
for p in docs:
    try:
        direcactual=os.getcwd()
        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\FINAMEX\Templates")

        rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\FINAMEX\\"+fecha+"\DicisFINAMEX\\"+p
        rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\FINAMEX\Templates\FINAMEX_Template.json"
        rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\FINAMEX\Templates\FINAMEX_TemplateFull.json"

        pdf = PyPDF2.PdfFileReader(rutapdf)


        full_temp = list()
        for n_pags in range((pdf.getNumPages())//2):
            with open(rutatemplate) as f:
                temp = json.load(f)
            z = temp.copy()
            [x.update({"page":x["page"]+2*n_pags}) for x in z]
            full_temp.extend(z)

        with open("FINAMEX_TemplateFull.json", "w") as f:
            json.dump(full_temp,f)

        data = tabula.read_pdf_with_template(rutapdf,rutafull)

        try:
            if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                tipoaux=101
            elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
                tipoaux=100
        except:
            tipoaux=None


        fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

        try:
            dia=fech.strip()[:2]
            mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
            año=fech.strip()[-4:].strip()
            auxfech=año+mes+dia
        except:
            auxfech=None

        hora=data[5].T.reset_index().T.dropna().iloc[1][1]
        try:
            if "-" in hora:
                auxhora=hora[hora.find("-")+1:].strip()
            elif "a" in hora:
                auxhora=hora[hora.find("a")+1:].strip()
        except:
            auxhora=None

        razon=' '.join( str(x) for x in list(data[1].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

        auxcali=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
        
        try:
            auxcali=auxcali.replace("f","").replace("S","").replace("(mex)","")
        except:
            auxcali=auxcali
            
        RES={
                                  "fondo": {
                                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                                        "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                                        "razon_social": {
                                            "valor": razon[:razon.find("Admin")].strip()
                                        },
                                        "fecha_autorizacion": {
                                            "valor": auxfech
                                        },
                                        "tipo": {
                                            "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                            "id": tipoaux
                                        },
                                        "administradora": {
                                            "nombre":razon[razon.find("por ")+3:].strip(),
                                            "valor": 22
                                        },
                                        "horizonte_inversion": {
                                            "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                        },
                                        "liquidez": {
                                            "valor": data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                        },
                                        "permanencia": {
                                            "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                        },
                                        "calificacion_mercado": {
                                            "valor": auxcali
                                        },
                                        "director": {
                                            "valor": fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                        },
                                        "vigente": 1,
                                        "categoria": {
                                            "valor": data[0].T.reset_index().T.dropna().iloc[:,1].dropna()[0].strip()
                                        },
                                        "horario_cierre": {
                                            "valor": auxhora,
                                        },
                                        "campos_faltantes": [],
                                        "serie":[
                                        ]
                                    }
                                }


        aux=[]
        for i in range((pdf.getNumPages())//2):
            padqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[1:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[1:,0].dropna())
            limites=" ".join(str(x) for x in data[(7*i)+2].T.reset_index().T.iloc[1:,2].dropna())

            try:
                comicliente = data[(7*i)+3].T.reset_index().T.iloc[0:,1][0].strip().replace("%","")
            except:
                comicliente=data[(7*i)+3].T.reset_index().T.iloc[0:,1][0]
            try:
                comifondo=data[(7*i)+4].T.reset_index().T.iloc[0:,1][0].strip().replace("%","")
            except:
                comifondo=data[(7*i)+4].T.reset_index().T.iloc[0:,1][0]
            
            if "." in padqui:
                adqui=list(data[(8*i)+1].T.reset_index().T.iloc[1:,1].dropna())[0].replace("\r"," ")[:list(data[(8*i)+1].T.reset_index().T.iloc[1:,1].dropna())[0].replace("\r"," ").find(".")]
            else:
                adqui=padqui

            aux.append({"nombre": serie,
                                        "monto_minimo": {
                                              "valor": limites
                                         },
                                         "monto_maximo": {
                                             "valor": None
                                                },
                                           "comision_cliente": {
                                              "valor": comicliente
                                                },
                                            "comision_fondo": {
                                              "valor": comifondo
                                                },
                                         "posibles_adquirientes": {
                                             "valor": adqui
                                                },
                                           "vigente":1
                                                ,
                                            "clave_BMV": RES["fondo"]["nombre"]+serie,
                                           "campos_faltantes": []
                                                })
        RES["fondo"]["serie"]=aux

        os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\FINAMEX\Jsons\\"+fecha)

        with open(fecha.replace("-","")+"_"+p[:p.find(".")]+".json", "w", encoding="ansi") as f:
            json.dump(RES, f, indent=2,ensure_ascii=False)
        print(f"success {p}")
    except Exception as e:
        print(f"error enfondo {p}")
        print(e)
        error.append(p)


# Value

In [41]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

fecha=str(datetime.date.today())
os.chdir("C:\\Users\\ortca\\Analysic_Nabla\\Proyectos\\INFOSEL\\ProcesoB\\Fondos_root\\VALUE\\Jsons")
#os.mkdir(fecha)
error=[]

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE F8.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F8.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F8.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   

    razon=" ".join(data[3].T.reset_index().T.iloc[:,1])

    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":razon,
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEF8"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEF8")
except:
    print("error fondo VALUEF8")
    error.append("VALUEF8")
    
try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE F1.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F1.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F1.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   



    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor":fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":" ".join(data[3].T.reset_index().T.iloc[:,1]),
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[3*i].T.reset_index().T.iloc[1:,1].dropna()).replace("Posibles Adquirentes","").strip()
        serie=" ".join(data2[3*1].T.reset_index().T.iloc[1:,0].dropna()).replace("Accionaria","").strip()
        limites=" ".join(data2[3*i].T.reset_index().T.iloc[1:,2].dropna()).replace("Inversión ($)","").strip()
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEF1"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUE F1")
except Exception as e:
    print("error fondo VALUE F1")
    print(e)
    error.append("VALUE F1")

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE F2.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F2.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F2.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   


    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":" ".join(data[3].T.reset_index().T.iloc[:,1]),
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEF2"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEF2")
except:
    print("error fondo VALUEF2")
    error.append("VALUEF2")
    
try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE F3.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F3.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F3.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   


    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":" ".join(data[3].T.reset_index().T.iloc[:,1]),
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEF3"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEF3")
except:
    print("error fondo VALUEF3")
    error.append("VALUEF3")

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE F4.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F4.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F4.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   


    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":" ".join(data[3].T.reset_index().T.iloc[:,1]),
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEF4"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEF4")
except:
    print("error fondo VALUEF4")
    error.append("VALUEF4")

try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE F7.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F7.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE F7.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   


    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":razon,
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEF7"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEF7")
except:
    print("error fondo VALUEF7")
    error.append("VALUEF7")


    
    #### checar horario V5
try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE V5.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE V5.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE V5.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   


    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":" ".join(data[3].T.reset_index().T.iloc[:,1]),
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[2].T.reset_index().T.iloc[:,1].dropna()[0][data[2].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEV5"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEV5")
except Except as e:
    print("error fondo VALUEV5")
    error.append("VALUEV5")
    
try:
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE")
    docs=os.listdir()
    error=[]
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\\VALUEFijos.csv")
    meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}

    direcactual=os.getcwd()
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates")

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\VALUE\\"+fecha+"\DicisVALUE\\"+ "VALUE V6.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE V6.json"
    rutatemplate2="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Templates\\VALUE V6.1.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)

    data = tabula.read_pdf_with_template(rutapdf,rutatemplate)
    data2 = tabula.read_pdf_with_template(rutapdf,rutatemplate2)

    try:
        if "variable" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=101
        elif "deuda" in data[0].T.reset_index().T.dropna().iloc[0][1].strip().lower():
            tipoaux=100
    except:
        tipoaux=None


    fech=data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())][1].strip()

    try:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=año+mes+dia
    except:
        auxfech=None   


    RES={
                      "fondo": {
                        "nombre": data[0].T.reset_index().T.dropna().iloc[2][1].strip(),
                         "url": fijos.iloc[0,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip(),
                          "razon_social": {
                            "valor": fijos.iloc[3,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                            "fecha_autorizacion": {
                             "valor": auxfech
                            },
                            "tipo": {
                              "valor": data[0].T.reset_index().T.dropna().iloc[0][1].strip(),
                                  "id": tipoaux
                                 },
                             "administradora": {
                               "nombre":" ".join(data[3].T.reset_index().T.iloc[:,1]),
                                 "valor": 24
                               },
                                "horizonte_inversion": {
                                 "valor": fijos.iloc[1,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                 },
                                  "liquidez": {
                                   "valor": data[1].T.reset_index().T.dropna().iloc[0][1].strip()
                                   },
                                     "permanencia": {
                                    "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.dropna().iloc[:,1]) if x != "Unnamed: 0" and type(x) != float).strip()
                                      },
                                     "calificacion_mercado": {
                                      "valor": data[0].T.reset_index().T.dropna().iloc[len(data[0].dropna())-1][1].strip()
                                        },
                                        "director": {
                                        "valor":  fijos.iloc[2,list(fijos.columns).index(data[0].T.reset_index().T.dropna().iloc[2][1].strip())].strip()
                                    },
                                    "vigente": 1,
                                     "categoria": {
                                         "valor": data[0].T.reset_index().T.dropna().iloc[1][1].strip().replace("\r"," ")
                                    },
                                    "horario_cierre": {
                                    "valor": data[1].T.reset_index().T.iloc[:,1].dropna()[0][data[1].T.reset_index().T.iloc[:,1].dropna()[0].find("a")+1:].strip(),
                                    },
                                     "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                             }

    aux=[]
    for i in range((pdf.getNumPages())//2):
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,1].dropna()[1:])
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,0].dropna()[1:])
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[1:,2].dropna()[1:])
        comicliente=data2[(3*i)+1].T.reset_index().T.iloc[:,1][0]
        comifondo=data2[(3*i)+2].T.reset_index().T.iloc[:,1][0]

        aux.append({"nombre": serie,
                                                "monto_minimo": {
                                                      "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": (RES["fondo"]["nombre"]+serie).replace(" ",""),
                                                   "campos_faltantes": []
                                                        })
    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\VALUE\Jsons\\"+fecha)

    with open(fecha.replace("-","")+"_"+"VALUEV6"+".json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print(f"success VALUEV6")
except Except as e:
    print("error fondo VALUEV6")
    error.append("VALUEV6")

success VALUEF8
success VALUE F1
success VALUEF2
success VALUEF3
success VALUEF4
success VALUEF7
success VALUEV5
success VALUEV6


# INVEX

In [ ]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}
try:

    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXCO.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXCO_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[3]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
            
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None


        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXCO.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXCO")
except:
    ptint("error INVEXCO")
    
try:

    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXCP.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXCP_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=101
        elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())+1]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-1]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
        
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None


        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXCP.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXCP")
except:
    print("error INVEXCP")
    

try:


    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXDO.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXDO_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[1:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None
    
    try:
        fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]
    except:
        fec="NA"
    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[3]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:2])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
            
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None


        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXDO.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXDO")
except:
    ptint("error INVEXDO")
    
try:
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXGU.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXGU_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=101
        elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())+1]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[4],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2:4])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
        
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXGU.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXGU")
except:
    printS("error INVEXGU")
    
try:

    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXIN.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXIN_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[1:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-1]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[3]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:2])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
            
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None


        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXIN.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXIN")
except:
    ptint("error INVEXIN")
    

try:


    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXLP.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXLP_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=101
        elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())+1]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[4],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2:4])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
        
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXLP.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXLP")
except:
    printS("error INVEXLP")
    

try:


    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXMX.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXMX_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[1:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[4]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre": list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
            
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None


        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXMX.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXMX")
except:
    ptint("error INVEXMX")
    

try:


    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXTK.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXTK_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[1:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=101
        elif "deuda" in list(data[1].T.reset_index().T.iloc[:,1].dropna())[0].strip().lower():
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[4]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre": list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[0],
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
            
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None


        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXTK.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXTK")
except:
    ptint("error INVEXTK")
    
try:


    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXMP.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXMP_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=101
        elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())+1]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-1]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
        
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXMP.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXMP")
except:
    print("error INVEXMP")
    

try:
    fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\\INVEXFijos.csv")
    fecha=str(datetime.date.today()).replace("-","")
    ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates\\"

    rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\INVEX\\"+str(datetime.date.today())+"\\DicisINVEX""\\"+"INVEXDX.pdf"
    rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEXDX_Template.json"
    rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\INVEX\\Templates\INVEX_TemplateFull.json"

    pdf = PyPDF2.PdfFileReader(rutapdf)


     #replicamos nuestro template para todas las series
    full_temp = list()
    for n_pags in range((pdf.getNumPages())//2):
        with open(rutatemplate) as f:
            temp = json.load(f)
        z = temp.copy()
        [x.update({"page":x["page"]+2*n_pags}) for x in z]
        full_temp.extend(z)
    
    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Templates")
            #guardamos el nuevo template
    with open("INVEX_TemplateFull.json", "w") as f:
        json.dump(full_temp,f)
    data = tabula.read_pdf_with_template(rutapdf,ruta+"INVEX_TemplateFull.json")
   

    razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

    try:
        if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=101
        elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()):
            tipoaux=100
    except:
        tipoaux=None

    fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())+1]

    try:
        if(len(fech.strip()[:2].strip())==1):
            dia="0"+fech.strip()[:2]
        else:
            dia=fech.strip()[:2]

        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()

        auxfech=(año+mes+dia).strip()
    except:
        auxfech=None

    auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-1]
    

    cierreaux=list(data[5].T.reset_index().T.iloc[:,1].dropna())[1]
    
    try:
        cierreaux=cierreaux[cierreaux.find("a")+1:].strip()
    except:
        cierreaux=cierreaux
    razon=" ".join(data[0].T.reset_index().T.iloc[1:,0].dropna())
    RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[:razon.lower().find("admin")].strip().replace(",","")
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 18
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[5].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[6].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
    aux=[]
    for i in range((pdf.getNumPages())//2):
        try:
            adqui=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,1].dropna())
            serie=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,0].dropna())
            limites=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            comicliente=data[(7*i)+3].T.reset_index().T.iloc[:,1][0].strip()
            comifondo=data[(7*1)+4].T.reset_index().T.iloc[:,1][0].strip()
        except:
            print("error")
        
        aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
        aux2=None
        
        if("-" in limites):
            aux1=limites[:limites.find("-")]
            aux2=limites[limites.find("-")+1:]
        elif ("–" in limites):
            aux1=limites[:limites.find("–")]
            aux2=limites[limites.find("–")+1:]
        else:
            aux1=" ".join(data[(7*i)+2].T.reset_index().T.iloc[2:,2].dropna())
            aux2=None

        aux.append({"nombre": serie,
                                "monto_minimo": {
                                          "valor": aux1
                                             },
                                             "monto_maximo": {
                                                 "valor": aux2
                                                    },
                                               "comision_cliente": {
                                                  "valor": comicliente
                                                    },
                                                "comision_fondo": {
                                                  "valor": comifondo
                                                    },
                                             "posibles_adquirientes": {
                                                 "valor": adqui
                                                    },
                                               "vigente":1
                                                    ,
                                                "clave_BMV": RES["fondo"]["nombre"]+serie,
                                               "campos_faltantes": []
                                                    })

    RES["fondo"]["serie"]=aux

    os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\INVEX\Jsons"+"\\"+str(datetime.date.today()))

    with open(fecha+"_"+"INVEXDX.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
    print("success INVEXDX")
except:
    print("error INVEXDX")

# GBM

In [90]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"DYNVAL.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\DYNVAL_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\DYNVAL_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"DYNVAL_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"DYNVAL_Template1.1.json")
   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
    año=fech.strip()[-4:].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[3].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[(2*i)].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[(2*i)].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[(2*i)].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"DYNVAL.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success DYNVAL")

error DYNVAL
error GBM103


In [62]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBM103.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM103_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"GBM103_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
    año=fech.strip()[-4:].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1]
    

cierreaux=list(data[6].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[6].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[5].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Series","").strip()
        limites=" ".join(data[2].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="1.2"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBM103.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBM103")

success GBM103


In [63]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBM104.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM104_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"GBM104_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
    año=fech.strip()[-4:].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1]
    

cierreaux=list(data[6].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[6].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[5].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Series","").strip()
        limites=" ".join(data[2].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="1.2"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBM104.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBM104")

success GBM104


In [65]:
## este tiene comisiones

import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBM105.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM105_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM105_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBM105_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBM105_Template1.1.json")
   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
    año=fech.strip()[-4:].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-2]
    

cierreaux=list(data[3].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[1:2,1].dropna()).lower()
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[(3*i)].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[(3*i)].T.reset_index().T.iloc[2:,0].dropna()).replace("Series","").strip()
        limites=" ".join(data2[(3*i)].T.reset_index().T.iloc[2:,2].dropna())
        comicliente=" ".join(data2[(3*i)+1].T.reset_index().T.iloc[:,1])
        comifondo=" ".join(data2[(3*i)+2].T.reset_index().T.iloc[:,1])
    except:
        print("error")
        
    try:
        comicliente=comicliente.replace("%","")
    except:
        print("error fondp")

        
    try:
        comifondo=comifondo.replace("%","")
    except:
        print("error comi")

    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": "N/A"
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBM105.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBM105")

success GBM105


In [66]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBM106.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM106_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM106_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBM106_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBM106_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
        año=fech.strip()[-5:-1].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[3].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="1.2"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBM106.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBM106")

success GBM106


In [67]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBM110.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM110_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM110_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBM110_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBM110_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[3].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[4],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[4])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:4,1].dropna()).lower()
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(data2[i].columns)-1].dropna())
        comicliente="0.75"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBM110.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBM110")

success GBM110


In [68]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBM112.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM112_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM112_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBM112_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBM112_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
        año=fech.strip()[-5:-1].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[3].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":"GBM Administradora de Activos, S.A. de C.V.,Sociedad Operadora de Fondos de Inversión",
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna()).lower()
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="0.30"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBM112.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBM112")

success GBM112


In [69]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMF2.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMF2_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMF2_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMF2_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMF2_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1]
    

cierreaux=list(data[3].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":"GBM Administradora de Activos, S.A. de C.V.,Sociedad Operadora de Fondos de Inversión",
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMF2.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMF2")

success GBMF2


In [73]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMF3.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMF3_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMF3_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMF3_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMF3_Template1.1.json")



try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
        año=fech.strip()[-5:-1].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1].replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
  

cierreaux=data[3].T.reset_index().T.iloc[:,0][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:razon.lower().find("admin")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[3].T.reset_index().T.dropna().loc["index",:][0].replace("Liquidez","").strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal[auxcal.find("/")-3:auxcal.find("/")+2]
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMF3.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMF3")

success GBMF3


In [72]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMGLB.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMGLB_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMGLB_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
    año=fech.strip()[-4:].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1]
    

cierreaux=list(data[6].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[6].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[5].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": list(data[1].T.reset_index().T.iloc[:,1].dropna())[2]
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Series","").strip()
        limites=" ".join(data[2].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="1.5"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMGLB.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMGLB")

success GBMGLB


In [75]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMPBOL.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMPBOL_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMPBOL_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())+2]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
    año=fech.strip()[-5:-1].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[4].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[5],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[5])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por ")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[5])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[4].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[5])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2:5])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data[2].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMPBOL.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMPBOL")

success GBMPBOL


In [81]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMPCON.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMPCON_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMPCON_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:3,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:3,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
    año=fech.strip()[-4:].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[4].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor":  " ".join(data[1].T.reset_index().T.iloc[:3,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find(" por ")+4:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[4].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data[2].T.reset_index().T.iloc[2:,2].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMPCON.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMPCON")

success GBMPCON


In [80]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMGUB.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMF3_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMGUB_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMGUB_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMGUB_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMGUB_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
        año=fech.strip()[-5:-1].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())].replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
    

cierreaux=data[3].T.reset_index().T.iloc[:,1][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal[auxcal.find("/")-3:auxcal.find("/")+2]  
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[3:,2].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")

    try:
        if ( len (limites)==0):
            limites=" ".join(data2[i].T.reset_index().T.iloc[2:,2].dropna())
    except:
        limites=" ".join(data2[i].T.reset_index().T.iloc[3:,2].dropna())
            
    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMGUB.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMGUB")

success GBMGUB


In [79]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMGUBL.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMGUBL_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMGUBL_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMGUBL_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMGUBL_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())]
    

cierreaux=data[3].T.reset_index().T.iloc[:,0][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":"GBM Administradora de Activos, S.A. de C.V.,Sociedad Operadora de Fondos de Inversión",
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMGUBL.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMGUBL")

success GBMGUBL


In [82]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMM3.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMM3_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMM3_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMM3_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMM3_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1]
    

cierreaux=data[3].T.reset_index().T.iloc[:,1][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":"GBM Administradora de Activos, S.A. de C.V.,Sociedad Operadora de Fondos de Inversión",
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMM3.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMM3")

success GBMM3


In [83]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMPAT.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMPAT_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMPAT_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMPAT_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMPAT_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-2] .replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
    

cierreaux=data[3].T.reset_index().T.iloc[:,1][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:razon.lower().find("admin")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal[auxcal.find("/")-3:auxcal.find("/")+2]  
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[1:2,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMPAT.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMPAT")

success GBMPAT


In [84]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMPDEU.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMPDEU_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMPDEU_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
    año=fech.strip()[-5:-1].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="N/A"
cierreaux=list(data[4].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[2],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find(" por ")+4:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[4].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[2])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:2])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMPDEU.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMPDEU")

success GBMPDEU


In [85]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"GBMRETO.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMRETO_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBMRETO_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"GBMRETO_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"GBMRETO_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1].replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
    

cierreaux=data[3].T.reset_index().T.iloc[:,1][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:razon.lower().find("admin")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal[auxcal.find("/")-3:auxcal.find("/")+2]  
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"GBMRETO.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success GBMRETO")

success GBMRETO


In [86]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"MPAGORF.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\MPAGORF_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\MPAGORF_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"MPAGORF_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"MPAGORF_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
        año=fech.strip()[-5:-1].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1].replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
    

cierreaux=data[3].T.reset_index().T.iloc[:,1][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:razon.lower().find("admin")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[2].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal[auxcal.find("/")-3:auxcal.find("/")+2]  
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"MPAGORF.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success MPAGORF")

success MPAGORF


In [87]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"REGIOB.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\REGIOBU_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"REGIOB_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
    año=fech.strip()[-5:-1].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[4].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find(" por ")+4:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[4].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor": " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"REGIOB.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success REGIOB")

success REGIOB


In [88]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"REGIOC.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\REGIOC_TemplateU.json"

pdf = PyPDF2.PdfFileReader(rutapdf)

data = tabula.read_pdf_with_template(rutapdf,ruta+"REGIOC_TemplateU.json")

   
razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(data[1].T.reset_index().T.dropna())]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]

    mes=meses[fech.strip()[2:-5].replace("de","").strip().lower()]
    año=fech.strip()[-5:-1].strip()
    auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal="NA"
    

cierreaux=list(data[4].T.reset_index().T.iloc[:,1].dropna())[1]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor": razon[razon.lower().find("la invers")+12:razon.lower().find("adminis")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:1,1].dropna()),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find(" por ")+4:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index( list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":data[4].T.reset_index().T.dropna().iloc[0][1].strip()
                                    },
                                    "permanencia": {
                                        "valor":' '.join( str(x) for x in list(data[3].T.reset_index().T.iloc[:3,1].dropna()) if x != "Unnamed: 0" and type(x) != float).strip()
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":  " ".join(list(data[1].T.reset_index().T.iloc[:,1].dropna())[1:3])
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }

aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data[2].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data[2].T.reset_index().T.iloc[2:,0].dropna()).replace("Serie","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="N/A"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })

RES["fondo"]["serie"]=aux

with open(fecha+"_"+"REGIOC.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success REGIOC")

success REGIOC


In [89]:
import os
import json
import pandas as pd
import tabula
import PyPDF2
import os 
import datetime

error=[]
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons")
#os.mkdir(str(datetime.date.today()))
os.chdir("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Jsons"+"\\"+str(datetime.date.today()))
meses={"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06","julio":"07","agosto":"08","septiembre":"09",
        "octubre":"10","noviembre":"11","diciembre":"12"}


fijos=pd.read_csv("C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\\GBMFijos.csv")
fecha=str(datetime.date.today()).replace("-","")
ruta="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\GBM\Templates\\"

rutapdf="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoA\FondosPDF_root\GBM\\"+str(datetime.date.today())+"\\DicisGBM""\\"+"SAURORT.pdf"
rutatemplate="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\SAURORT_Template1.json"
rutaaux="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\SAURORT_Template1.1.json"
rutafull="C:\\Users\ortca\Analysic_Nabla\Proyectos\INFOSEL\ProcesoB\Fondos_root\\GBM\\Templates\GBM_TemplateFull.json"

pdf = PyPDF2.PdfFileReader(rutapdf)


data = tabula.read_pdf_with_template(rutapdf,ruta+"SAURORT_Template1.json")
data2 = tabula.read_pdf_with_template(rutapdf,ruta+"SAURORT_Template1.1.json")

razon=' '.join( str(x) for x in list(data[0].T.reset_index().T.dropna().iloc[:,0]) if x != "Unnamed: 0" and type(x) != float).strip()

try:
    if "variable" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=101
    elif "deuda" in " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower():
        tipoaux=100
except:
    tipoaux=None

fech=list(data[1].T.reset_index().T.iloc[:,1].dropna())[len(list(data[1].T.reset_index().T.iloc[:,1].dropna()))-1]

try:
    if(len(fech.strip()[:2].strip())==1):
        dia="0"+fech.strip()[:2]
    else:
        dia=fech.strip()[:2]
        mes=meses[fech.strip()[2:-4].replace("de","").strip().lower()]
        año=fech.strip()[-4:].strip()
        auxfech=(año+mes+dia).strip()
except:
    auxfech=None

auxcal=data[1].T.reset_index().T.iloc[:,1].dropna()[len(data[1].dropna())-1]
    

cierreaux=data[3].T.reset_index().T.iloc[:,1][0]
    
try:
    cierreaux=cierreaux[cierreaux.find("–")+1:].strip()
except:
    cierreaux=cierreaux
    
RES={
                              "fondo": {
                                    "nombre":list(data[1].T.reset_index().T.iloc[:,1].dropna())[3],
                                    "url":fijos.iloc[0,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip(),
                                    "razon_social": {
                                        "valor":  razon[razon.lower().find("la invers")+12:razon.lower().find("administ")].replace(",","").strip()
                                    },
                                    "fecha_autorizacion": {
                                        "valor":auxfech
                                    },
                                    "tipo": {
                                        "valor": " ".join(data[1].T.reset_index().T.iloc[:2,1].dropna()).lower(),
                                        "id": tipoaux
                                    },
                                    "administradora": {
                                        "nombre":razon[razon.lower().find("por")+3:].strip().replace(",","").strip(),
                                        "valor": 14
                                    },
                                    "horizonte_inversion": {
                                        "valor": fijos.iloc[1,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "liquidez": {
                                        "valor":" ".join(data[3].T.reset_index().T.dropna().iloc[:1,1])
                                    },
                                    "permanencia": {
                                        "valor":"N/A"
                                    },
                                    "calificacion_mercado": {
                                        "valor": auxcal.replace("S","").replace("f","").replace("(mex)","").replace("s","").replace("F","")
                                    },
                                    "director": {
                                        "valor": fijos.iloc[2,list(fijos.columns).index(list(data[1].T.reset_index().T.iloc[:,1].dropna())[3])].strip()
                                    },
                                    "vigente": 1,
                                    "categoria": {
                                        "valor":   " ".join(data[1].T.reset_index().T.iloc[2:3,1].dropna())
                                    },
                                    "horario_cierre": {
                                        "valor": cierreaux,
                                    },
                                    "campos_faltantes": [],
                                    "serie":[
                                    ]
                                }
                            }
aux=[]
for i in range((pdf.getNumPages())//2):
    try:
        adqui=" ".join(data2[i].T.reset_index().T.iloc[2:,1].dropna())
        serie=" ".join(data2[i].T.reset_index().T.iloc[2:,0].dropna()).replace("Series","").strip()
        limites=" ".join(data2[i].T.reset_index().T.iloc[2:,len(list(data2[i].columns))-1].dropna())
        comicliente="0.75"
        comifondo="N/A"
    except:
        print("error")


    aux.append({"nombre": serie,
                                    "monto_minimo": {
                                              "valor": limites
                                                 },
                                                 "monto_maximo": {
                                                     "valor": None
                                                        },
                                                   "comision_cliente": {
                                                      "valor": comicliente
                                                        },
                                                    "comision_fondo": {
                                                      "valor": comifondo
                                                        },
                                                 "posibles_adquirientes": {
                                                     "valor": adqui
                                                        },
                                                   "vigente":1
                                                        ,
                                                    "clave_BMV": RES["fondo"]["nombre"]+serie,
                                                   "campos_faltantes": []
                                                        })
RES["fondo"]["serie"]=aux

with open(fecha+"_"+"SAURORT.json", "w", encoding="ansi") as f:
        json.dump(RES, f, indent=2,ensure_ascii=False)
print("success SAURORT")

success SAURORT
